In [1]:
import os
import random
import numpy as np
import torch

def set_seed(seed: int = 42):
    # Python built-in random
    random.seed(seed)
    
    # Numpy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    
    # For reproducibility
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Hash-based operations
    os.environ["PYTHONHASHSEED"] = str(seed)

# Call this once at the very top of your notebook
set_seed(42)
import random

# --- TAMBAHKAN BLOK INI UNTUK REPRODUCIBILITY ---
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # jika menggunakan multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [ ]:
# ===================================================================
# === 1. IMPORT LIBRARY ===
# ===================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import os
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report, accuracy_score
import cv2
import timm
from pathlib import Path
from tqdm.auto import tqdm
import imagehash

# ===================================================================
# === 2. KONFIGURASI DAN SETUP (TIDAK DIUBAH) ===
# ===================================================================
# Direktori
BASE_DIR = os.getcwd()  # Menggunakan current working directory
TRAIN_DIR = os.path.join(BASE_DIR, "dataset fixx", "train")
TEST_DIR = os.path.join(BASE_DIR, "dataset fixx", "test", "test")

# Konfigurasi Model dan Training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 518
BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count()

# --- HYPERPARAMETER UNTUK GRADUAL UNFREEZING ---
EPOCHS_S1 = 3
LR_S1 = 3e-4
EPOCHS_S2 = 5
LR_S2 = 5e-5
EPOCHS_S3 = 12
LR_S3 = 1.59e-06 # <-- Anda mungkin perlu MENYESUAIKAN ini untuk model baru

# Statistik ImageNet
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

print(f"Menggunakan device: {DEVICE}")
print(f"Ukuran gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# ===================================================================
# === 3. FUNGSI DAN KELAS HELPER (TIDAK DIUBAH) ===
# ===================================================================
def convert_path_to_df(dataset, is_test=False):
    image_dir = Path(dataset)
    
    # 1. Tentukan ekstensi gambar yang valid (dalam huruf kecil)
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff'}

    # 2. Ambil semua file path terlebih dahulu
    all_filepaths = list(image_dir.glob(r'**/*.*'))
    
    # 3. Filter daftar file, hanya ambil yang ekstensinya ada di set
    filepaths = [p for p in all_filepaths if p.suffix.lower() in image_extensions]
    
    # Cek jika tidak ada gambar yang ditemukan
    if not filepaths:
        print(f"Peringatan: Tidak ada file gambar yang ditemukan di {image_dir}")
        if is_test:
            return pd.DataFrame(columns=['Filepath'])
        else:
            return pd.DataFrame(columns=['Filepath', 'Label'])

    # 4. Sisa logika Anda tetap sama
    if is_test:
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        return pd.DataFrame({'Filepath': filepaths})
    else:
        labels = [p.parts[-2] for p in filepaths]
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        labels = pd.Series(labels, name='Label')
        return pd.concat([filepaths, labels], axis=1)

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column=None, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform: image = self.transform(image)
        if self.label_column:
            label = self.dataframe.iloc[idx][self.label_column]
            return image, torch.tensor(label, dtype=torch.long)
        return image
class DualTransformDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column, transform_main, transform_extra):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform_main = transform_main
        self.transform_extra = transform_extra

    def __len__(self):
        # Gandakan ukuran dataset (dua versi per gambar)
        return len(self.dataframe) * 2

    def __getitem__(self, idx):
        # Tentukan apakah pakai augmentasi utama atau tambahan
        base_idx = idx // 2
        use_extra = idx % 2 == 1

        row = self.dataframe.iloc[base_idx]
        img_path = row[self.image_column]
        label = torch.tensor(row[self.label_column], dtype=torch.long)

        image = Image.open(img_path).convert('RGB')

        if use_extra:
            image = self.transform_extra(image)
        else:
            image = self.transform_main(image)

        return image, label

class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    def __call__(self, img):
        img_np = np.array(img); img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2Lab)
        l, a, b = cv2.split(img_lab); l_clahe = self.clahe.apply(l)
        img_lab_clahe = cv2.merge((l_clahe, a, b))
        img_rgb_clahe = cv2.cvtColor(img_lab_clahe, cv2.COLOR_Lab2RGB)
        return Image.fromarray(img_rgb_clahe)

class TestDataset(Dataset):
    def __init__(self, dataframe, image_column, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# ===================================================================
# === 4. PERSIAPAN DATA DENGAN STRATEGI "VALIDASI BERSIH" (TIDAK DIUBAH) ===
# ===================================================================
def get_phash(filepath):
    try:
        with Image.open(filepath) as img: return imagehash.phash(img)
    except Exception: return None

# --- Langkah 4.1: Memuat dan memfilter data training secara manual ---
train_df = convert_path_to_df(TRAIN_DIR)

print(f"Jumlah data training before filtering manual: {len(train_df)}")


# --- Langkah 4.2: Identifikasi Kebocoran dan Pisahkan Data ---
test_df = convert_path_to_df(TEST_DIR, is_test=True)
print("\n" + "="*50)
print("Mengidentifikasi kebocoran data (train vs test)...")
tqdm.pandas(desc="Menghitung Hash Data Test")
test_hashes = set(test_df['Filepath'].progress_apply(get_phash))
test_hashes.discard(None)

tqdm.pandas(desc="Menghitung Hash Data Train")
train_df['hash'] = train_df['Filepath'].progress_apply(get_phash)

train_df['is_leak'] = train_df['hash'].isin(test_hashes)
print(f"Ditemukan {train_df['is_leak'].sum()} gambar di training set yang identik dengan gambar di test set.")

print("\nMenerapkan strategi 'Validasi Bersih'...")
leaked_df = train_df[train_df['is_leak']].copy()
clean_df = train_df[~train_df['is_leak']].copy()

# --- Langkah 4.3: Pemetaan Label dan Split Data ---
label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14
}

clean_df['Label'] = clean_df['Label'].map(label_mapping)
leaked_df['Label'] = leaked_df['Label'].map(label_mapping)

val_split = pd.DataFrame()
if not clean_df.empty:
    try:
        clean_train_split, val_split = train_test_split(
            clean_df, test_size=0.2, random_state=42, stratify=clean_df['Label'])
    except ValueError:
        print("Peringatan: Gagal stratify, menggunakan split biasa.")
        clean_train_split, val_split = train_test_split(clean_df, test_size=0.2, random_state=42)
else:
    clean_train_split = clean_df

train_split = pd.concat([clean_train_split, leaked_df], ignore_index=True)
train_split.drop(columns=['hash', 'is_leak'], inplace=True)
val_split.drop(columns=['hash', 'is_leak'], inplace=True)
print(f"Ukuran set training final (sisa bersih + semua bocor): {len(train_split)}")
print(f"Ukuran set validasi murni (hanya dari data bersih): {len(val_split)}")
print("="*50 + "\n")

# --- Langkah 4.4: Lanjutkan dengan pipeline seperti biasa ---
train_transform = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
# Transform tambahan: rotasi 90° kiri/kanan
train_transform_extra = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([
        transforms.RandomRotation((90, 90)),
        transforms.RandomRotation((-90, -90))
    ]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

train_dataset = DualTransformDataset(
    train_split,
    image_column='Filepath',
    label_column='Label',
    transform_main=train_transform,
    transform_extra=train_transform_extra
)

val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)
# PASTIKAN SEMUA LABEL BERTIPE INTEGER
train_split["Label"] = train_split["Label"].astype(int)

# Kode Anda yang sebelumnya
class_weights = compute_class_weight("balanced", classes=np.unique(train_split["Label"]), y=train_split["Label"])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)


# ===================================================================
# === 5. DEFINISI MODEL (DIUBAH) ===
# ===================================================================

# --- MODEL BARU: SINGLE DINOV2 ---
class SingleDINOv2Model(nn.Module):
    def __init__(self, num_classes=5): # num_classes akan diisi oleh len(label_mapping)
        super().__init__()
        # timm akan otomatis menambahkan head klasifikasi untuk kita
        self.backbone = timm.create_model(
            "vit_base_patch14_dinov2", 
            pretrained=True, 
            num_classes=num_classes 
        )
    def forward(self, x):
        return self.backbone(x)

# --- MODEL LAMA (Untuk referensi) ---
# class FusionDINOv2(nn.Module):
#     def __init__(self, num_classes=5):
#         super().__init__()
#         self.dinov2 = timm.create_model("vit_base_patch14_dinov2", pretrained=True, num_classes=0)
#         self.convnext = timm.create_model("convnext_tiny", pretrained=True, num_classes=0)
#         fusion_dim = self.dinov2.num_features + self.convnext.num_features
#         self.classifier = nn.Sequential(
#             nn.Linear(fusion_dim, 512),
#             nn.LayerNorm(512),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(512, num_classes)
#         )
#     def forward(self, x):
#         feat_dino = self.dinov2(x)
#         feat_conv = self.convnext(x)
#         feat_combined = torch.cat((feat_dino, feat_conv), dim=1)
#         out = self.classifier(feat_combined)
#         return out
        
class SingleSwinModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)


Menggunakan device: cuda
Ukuran gambar: 518x518, Batch Size: 8
Jumlah data training before filtering manual: 4052

Mengidentifikasi kebocoran data (train vs test)...


Menghitung Hash Data Test:   0%|          | 0/2057 [00:00<?, ?it/s]

In [ ]:
# ===================================================================
# === 1. TAMBAHKAN IMPORT INI DI BAGIAN ATAS SKRIP ANDA ===
# ===================================================================
from timm.data.mixup import Mixup
# (Asumsi impor lain seperti torch, timm, dll sudah ada)
# ...
import torch.nn.functional as F

###################################################################################
### BLOK B: MELATIH MODEL EVA TRANSFORMER (DENGAN MIXUP/CUTMIX)
###################################################################################
print("\n" + "#"*80)
print("### MEMULAI BLOK B: TRAINING MODEL EVA TRANSFORMER (MIXUP/CUTMIX) ###")
print("#"*80 + "\n")

# B.1. Konfigurasi khusus untuk Model EVA
IMG_SIZE = 224
BATCH_SIZE = 16
BEST_MODEL_NAME_EVA = "best_eva_standalone.pth" 
print(f"Menggunakan Ukuran Gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# B.2. Buat Ulang Transforms dan DataLoaders
# (Tidak ada perubahan di sini, kode Anda sudah benar)
train_transform_main_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
train_transform_extra_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([transforms.RandomRotation((90, 90)), transforms.RandomRotation((-90, -90))]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform_224 = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
train_dataset = DualTransformDataset(train_split, 'Filepath', 'Label', train_transform_main_224, train_transform_extra_224)
val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform_224)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)


# B.3. Inisialisasi Model
class SingleEvaModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = timm.create_model("eva02_base_patch14_224", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

model = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE)
print(f"Menggunakan model: eva02_base_patch14_224")

criterion = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.1)

# --- TAMBAHKAN INISIALISASI MIXUP ---
print("Menginisialisasi Mixup/CutMix...")
mixup_fn = Mixup(
    mixup_alpha=0.8,    # Nilai standar timm
    cutmix_alpha=1.0,   # Nilai standar timm
    prob=1.0,           # Terapkan salah satu (mixup/cutmix) pada SETIAP batch
    switch_prob=0.5,    # 50% Mixup, 50% CutMix
    mode='batch',
    label_smoothing=0.1,# Selaras dengan criterion Anda
    num_classes=len(label_mapping)
)
# ------------------------------------

scaler = torch.amp.GradScaler()
best_f1 = 0.0
best_acc = 0.0

# B.4. Lakukan Training (3 Tahap disesuaikan untuk EVA)
# --- TAHAP 1 (Tidak Berubah) ---
print("\n" + "="*50 + "\nTAHAP 1 (EVA): Melatih Classifier Head\n" + "="*50)
for param in model.backbone.parameters(): param.requires_grad = False
for param in model.backbone.head.parameters(): param.requires_grad = True 
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1)
for epoch in range(EPOCHS_S1):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S1 EVA Epoch {epoch+1}/{EPOCHS_S1}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # --- PERBAIKAN: Jangan gunakan Mixup di Tahap 1 ---
        # Kita ingin head belajar pemetaan sederhana terlebih dahulu
        
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            # CrossEntropyLoss TIDAK bisa menangani label [B] dan smoothing
            # Jadi kita harus membuatnya one-hot manual untuk Tahap 1 & 2
            labels_one_hot = F.one_hot(labels, num_classes=len(label_mapping)).float()
            outputs = model(images); loss = criterion(outputs, labels_one_hot)
            
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()

# --- TAHAP 2 (Tidak Berubah) ---
print("\n" + "="*50 + "\nTAHAP 2 (EVA): Melatih Head + Blok Atas\n" + "="*50)
for param in model.backbone.blocks[-1].parameters(): param.requires_grad = True
for param in model.backbone.norm.parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2)
for epoch in range(EPOCHS_S2):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S2 EVA Epoch {epoch+1}/{EPOCHS_S2}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            # Gunakan one-hot yang sama
            labels_one_hot = F.one_hot(labels, num_classes=len(label_mapping)).float()
            outputs = model(images); loss = criterion(outputs, labels_one_hot)
            
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        
# --- TAHAP 3 (DIMODIFIKASI) ---

# --- REKOMENDASI: TAMBAHKAN EPOCH KARENA MIXUP ---
EPOCHS_S3_MIXUP = 25 # Coba 25-30 epoch, karena Mixup butuh waktu lebih lama
print(f"TAHAP 3 akan berjalan selama {EPOCHS_S3_MIXUP} epochs (lebih lama untuk Mixup/CutMix)")
# -------------------------------------------------

LR_S3_EVA = 1e-5 
print("\n" + "="*50 + "\nTAHAP 3 (EVA): Fine-tuning Seluruh Model (dengan Mixup/CutMix)\n" + "="*50)
for param in model.parameters(): param.requires_grad = True
optimizer = optim.AdamW(model.parameters(), lr=LR_S3_EVA, weight_decay=1e-4)

# --- SESUAIKAN T_max DENGAN EPOCH BARU ---
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS_S3_MIXUP, # <-- GANTI
    eta_min=1e-6    
)
# ----------------------------------------

for epoch in range(EPOCHS_S3_MIXUP): # <-- GANTI
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"S3 EVA Epoch {epoch+1}/{EPOCHS_S3_MIXUP} (Train)") # <-- GANTI
    for images, labels in train_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        # --- TERAPKAN MIXUP/CUTMIX DI SINI ---
        images, labels = mixup_fn(images, labels)
        # 'labels' sekarang menjadi [B, N_CLASSES] (soft labels)
        # ------------------------------------
        
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images)
            # 'criterion' bisa langsung menangani 'labels' soft [B, N_CLASSES]
            loss = criterion(outputs, labels) 
            
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer); scaler.update()
        # scheduler.step() # <-- HAPUS DARI SINI
        train_loss += loss.item()
    
    # --- PINDAHKAN SCHEDULER.STEP() KE SINI ---
    # Panggil scheduler HANYA sekali per epoch
    scheduler.step()
    # ---------------------------------------

    model.eval()
    val_loss, all_preds, all_labels = 0.0, [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            with torch.amp.autocast(device_type = 'cuda'):
                # Untuk validasi, kita gunakan label asli (hard labels)
                # jadi kita TIDAK perlu one-hot
                outputs = model(images)
                loss = criterion(outputs, labels) # <-- 'labels' di sini [B]
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy()); all_labels.extend(labels.cpu().numpy())
            
    f1 = f1_score(all_labels, all_preds, average="macro")
    acc = accuracy_score(all_labels, all_preds) 
    
    # Opsi: Tampilkan LR saat ini untuk memverifikasi
    current_lr = optimizer.param_groups[0]['lr']
    print(f"\nEpoch [{epoch+1}/{EPOCHS_S3_MIXUP}] | LR: {current_lr:.2e} | Val F1: {f1:.4f}| Val acc: {acc:.4f}")
    
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), BEST_MODEL_NAME_EVA) 
        print(f"✅ Model EVA disimpan ke {BEST_MODEL_NAME_EVA} (F1 terbaik baru: {best_acc:.4f})") 

print(f"\nTraining Model EVA Selesai. Bobot terbaik disimpan di {BEST_MODEL_NAME_EVA}") 



################################################################################
### MEMULAI BLOK B: TRAINING MODEL EVA TRANSFORMER (MIXUP/CUTMIX) ###
################################################################################

Menggunakan Ukuran Gambar: 224x224, Batch Size: 16
Menggunakan model: eva02_base_patch14_224
Menginisialisasi Mixup/CutMix...

TAHAP 1 (EVA): Melatih Classifier Head


S1 EVA Epoch 1/3:   0%|          | 0/408 [00:00<?, ?it/s]

S1 EVA Epoch 2/3:   0%|          | 0/408 [00:00<?, ?it/s]

S1 EVA Epoch 3/3:   0%|          | 0/408 [00:00<?, ?it/s]


TAHAP 2 (EVA): Melatih Head + Blok Atas


...


Epoch [23/25] | LR: 1.14e-06 | Val F1: 0.9484| Val acc: 0.9484


S3 EVA Epoch 24/25 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [24/25] | LR: 1.04e-06 | Val F1: 0.9495| Val acc: 0.9497


S3 EVA Epoch 25/25 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [25/25] | LR: 1.00e-06 | Val F1: 0.9522| Val acc: 0.9522

Training Model EVA Selesai. Bobot terbaik disimpan di best_eva_standalone.pth


In [ ]:
# ==================================================================
# === 1. IMPORT LIBRARY ===
# ===================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import os
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, classification_report, accuracy_score
import cv2
import timm
from pathlib import Path
from tqdm.auto import tqdm
import imagehash

# ===================================================================
# === 2. KONFIGURASI DAN SETUP (TIDAK DIUBAH) ===
# ===================================================================
# Direktori
TRAIN_DIR = "/kaggle/input/newdatasets/train"
TEST_DIR = "/kaggle/input/newdatasets/test/test"

# Konfigurasi Model dan Training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 518
BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count()

# --- HYPERPARAMETER UNTUK GRADUAL UNFREEZING ---
EPOCHS_S1 = 3
LR_S1 = 3e-4
EPOCHS_S2 = 5
LR_S2 = 5e-5
EPOCHS_S3 = 12
LR_S3 = 1.59e-06

# Statistik ImageNet
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

print(f"Menggunakan device: {DEVICE}")
print(f"Ukuran gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# ===================================================================
# === 3. FUNGSI DAN KELAS HELPER (TIDAK DIUBAH) ===
# ===================================================================
def convert_path_to_df(dataset, is_test=False):
    image_dir = Path(dataset)
    
    # 1. Tentukan ekstensi gambar yang valid (dalam huruf kecil)
    # Menggunakan set ( {} ) agar pencarian lebih cepat
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff'}

    # 2. Ambil semua file path terlebih dahulu
    all_filepaths = list(image_dir.glob(r'**/*.*'))
    
    # 3. Filter daftar file, hanya ambil yang ekstensinya ada di set
    # p.suffix akan mengambil ekstensi (mis: ".JPG")
    # .lower() mengubahnya jadi huruf kecil (mis: ".jpg")
    filepaths = [p for p in all_filepaths if p.suffix.lower() in image_extensions]
    
    # Cek jika tidak ada gambar yang ditemukan
    if not filepaths:
        print(f"Peringatan: Tidak ada file gambar yang ditemukan di {image_dir}")
        if is_test:
            return pd.DataFrame(columns=['Filepath'])
        else:
            return pd.DataFrame(columns=['Filepath', 'Label'])

    # 4. Sisa logika Anda tetap sama
    if is_test:
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        return pd.DataFrame({'Filepath': filepaths})
    else:
        labels = [p.parts[-2] for p in filepaths]
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        labels = pd.Series(labels, name='Label')
        return pd.concat([filepaths, labels], axis=1)

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column=None, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform: image = self.transform(image)
        if self.label_column:
            label = self.dataframe.iloc[idx][self.label_column]
            return image, torch.tensor(label, dtype=torch.long)
        return image
class DualTransformDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column, transform_main, transform_extra):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform_main = transform_main
        self.transform_extra = transform_extra

    def __len__(self):
        # Gandakan ukuran dataset (dua versi per gambar)
        return len(self.dataframe) * 2

    def __getitem__(self, idx):
        # Tentukan apakah pakai augmentasi utama atau tambahan
        base_idx = idx // 2
        use_extra = idx % 2 == 1

        row = self.dataframe.iloc[base_idx]
        img_path = row[self.image_column]
        label = torch.tensor(row[self.label_column], dtype=torch.long)

        image = Image.open(img_path).convert('RGB')

        if use_extra:
            image = self.transform_extra(image)
        else:
            image = self.transform_main(image)

        return image, label

class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    def __call__(self, img):
        img_np = np.array(img); img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2Lab)
        l, a, b = cv2.split(img_lab); l_clahe = self.clahe.apply(l)
        img_lab_clahe = cv2.merge((l_clahe, a, b))
        img_rgb_clahe = cv2.cvtColor(img_lab_clahe, cv2.COLOR_Lab2RGB)
        return Image.fromarray(img_rgb_clahe)

class TestDataset(Dataset):
    def __init__(self, dataframe, image_column, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# ===================================================================
# === 4. PERSIAPAN DATA DENGAN STRATEGI "VALIDASI BERSIH" (DIUBAH) ===
# ===================================================================
def get_phash(filepath):
    try:
        with Image.open(filepath) as img: return imagehash.phash(img)
    except Exception: return None

# --- Langkah 4.1: Memuat dan memfilter data training secara manual ---
train_df = convert_path_to_df(TRAIN_DIR)

print(f"Jumlah data training sebelum filtering manual: {len(train_df)}")


# --- Langkah 4.2: Identifikasi Kebocoran dan Pisahkan Data ---
test_df = convert_path_to_df(TEST_DIR, is_test=True)
print("\n" + "="*50)
print("Mengidentifikasi kebocoran data (train vs test)...")
tqdm.pandas(desc="Menghitung Hash Data Test")
test_hashes = set(test_df['Filepath'].progress_apply(get_phash))
test_hashes.discard(None)

tqdm.pandas(desc="Menghitung Hash Data Train")
train_df['hash'] = train_df['Filepath'].progress_apply(get_phash)

train_df['is_leak'] = train_df['hash'].isin(test_hashes)
print(f"Ditemukan {train_df['is_leak'].sum()} gambar di training set yang identik dengan gambar di test set.")

print("\nMenerapkan strategi 'Validasi Bersih'...")
leaked_df = train_df[train_df['is_leak']].copy()
clean_df = train_df[~train_df['is_leak']].copy()

# --- Langkah 4.3: Pemetaan Label dan Split Data ---
label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14
}

clean_df['Label'] = clean_df['Label'].map(label_mapping)
leaked_df['Label'] = leaked_df['Label'].map(label_mapping)

val_split = pd.DataFrame()
if not clean_df.empty:
    try:
        clean_train_split, val_split = train_test_split(
            clean_df, test_size=0.2, random_state=42, stratify=clean_df['Label'])
    except ValueError:
        print("Peringatan: Gagal stratify, menggunakan split biasa.")
        clean_train_split, val_split = train_test_split(clean_df, test_size=0.2, random_state=42)
else:
    clean_train_split = clean_df

train_split = pd.concat([clean_train_split, leaked_df], ignore_index=True)
train_split.drop(columns=['hash', 'is_leak'], inplace=True)
val_split.drop(columns=['hash', 'is_leak'], inplace=True)
print(f"Ukuran set training final (sisa bersih + semua bocor): {len(train_split)}")
print(f"Ukuran set validasi murni (hanya dari data bersih): {len(val_split)}")
print("="*50 + "\n")

# --- Langkah 4.4: Lanjutkan dengan pipeline seperti biasa ---
train_transform = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
# Transform tambahan: rotasi 90° kiri/kanan
train_transform_extra = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([
        transforms.RandomRotation((90, 90)),
        transforms.RandomRotation((-90, -90))
    ]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

train_dataset = DualTransformDataset(
    train_split,
    image_column='Filepath',
    label_column='Label',
    transform_main=train_transform,
    transform_extra=train_transform_extra
)

val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)
# PASTIKAN SEMUA LABEL BERTIPE INTEGER
train_split["Label"] = train_split["Label"].astype(int)

# Kode Anda yang sebelumnya
class_weights = compute_class_weight("balanced", classes=np.unique(train_split["Label"]), y=train_split["Label"])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)


# ===================================================================
# === 5. DEFINISI MODEL (TIDAK DIUBAH) ===
# ===================================================================
class FusionDINOv2(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.dinov2 = timm.create_model("vit_base_patch14_dinov2", pretrained=True, num_classes=0)
        self.convnext = timm.create_model("convnext_tiny", pretrained=True, num_classes=0)
        fusion_dim = self.dinov2.num_features + self.convnext.num_features
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        feat_dino = self.dinov2(x)
        feat_conv = self.convnext(x)
        feat_combined = torch.cat((feat_dino, feat_conv), dim=1)
        out = self.classifier(feat_combined)
        return out
class SingleSwinModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

# # ===================================================================
# # === 6. INISIALISASI MODEL, LOSS, DAN SCALER (TIDAK DIUBAH) ===
# # ===================================================================
model = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
scaler = torch.amp.GradScaler()
best_f1 = 0.0
best_acc = 0.0

# ==================================================================================
# === 7. STRATEGI TRAINING: GRADUAL UNFREEZING (TIDAK DIUBAH) ===
#==================================================================================
# # --- TAHAP 1: Latih hanya Classifier Head ---
# print("\n" + "="*50 + "\nTAHAP 1: Melatih Classifier Head\n" + "="*50)
# for param in model.dinov2.parameters(): param.requires_grad = False
# for param in model.convnext.parameters(): param.requires_grad = False
# for param in model.classifier.parameters(): param.requires_grad = True
# optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1)
# for epoch in range(EPOCHS_S1):
#     model.train()
#     progress_bar = tqdm(train_loader, desc=f"S1 Epoch {epoch+1}/{EPOCHS_S1}")
#     for images, labels in progress_bar:
#         images, labels = images.to(DEVICE), labels.to(DEVICE)
#         optimizer.zero_grad()
#         with torch.amp.autocast(device_type = 'cuda'):
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

# # --- TAHAP 2: Latih Head + Setengah Atas DINOv2 ---
# print("\n" + "="*50 + "\nTAHAP 2: Melatih Head + Setengah Atas DINOv2\n" + "="*50)
# total_blocks = len(model.dinov2.blocks)
# for i in range(total_blocks // 2, total_blocks):
#     for param in model.dinov2.blocks[i].parameters():
#         param.requires_grad = True
# optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2)
# for epoch in range(EPOCHS_S2):
#     model.train()
#     progress_bar = tqdm(train_loader, desc=f"S2 Epoch {epoch+1}/{EPOCHS_S2}")
#     for images, labels in progress_bar:
#         images, labels = images.to(DEVICE), labels.to(DEVICE)
#         optimizer.zero_grad()
#         with torch.amp.autocast(device_type = 'cuda'):
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()



# print("\n" + "="*50 + "\nTAHAP 3: Fine-tuning Seluruh Model\n" + "="*50)
# for param in model.parameters(): param.requires_grad = True
# optimizer = optim.AdamW(model.parameters(), weight_decay=1e-4)
# scheduler = scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#     optimizer,
#     T_max=EPOCHS_S3,   # jumlah iterasi (biasanya epoch)
#     eta_min=1e-6       # nilai LR minimum saat akhir siklus
# )

# for epoch in range(EPOCHS_S3):
#     model.train()
#     train_loss = 0.0
#     train_bar = tqdm(train_loader, desc=f"S3 Epoch {epoch+1}/{EPOCHS_S3} (Train)")
#     for images, labels in train_bar:
#         images, labels = images.to(DEVICE), labels.to(DEVICE)
#         optimizer.zero_grad()
#         with torch.amp.autocast(device_type = 'cuda'):
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#         scaler.scale(loss).backward()
#         nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
#         scaler.step(optimizer)
#         scaler.update()
#         scheduler.step()
#         train_loss += loss.item()
    
#     model.eval()
#     val_loss, all_preds, all_labels = 0.0, [], []
#     val_bar = tqdm(val_loader, desc=f"S3 Epoch {epoch+1}/{EPOCHS_S3} (Val)")
#     with torch.no_grad():
#         for images, labels in val_bar:
#             images, labels = images.to(DEVICE), labels.to(DEVICE)
#             with torch.amp.autocast(device_type = 'cuda'):
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
#             val_loss += loss.item()
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
            
#     avg_train_loss = train_loss / len(train_loader)
#     avg_val_loss = val_loss / len(val_loader)
#     f1 = f1_score(all_labels, all_preds, average="macro")
#     acc = accuracy_score(all_labels, all_preds) 
#     print(f"\nEpoch [{epoch+1}/{EPOCHS_S3}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Macro F1: {f1:.4f} | acc= {acc:.4f}")
    
#     report = classification_report(all_labels, all_preds, target_names=list(label_mapping.keys()))
#     print("\n--- Laporan Klasifikasi Validasi ---")
#     print(report)
    
    
#     if acc > best_acc:
#         best_acc = acc
#         torch.save(model.state_dict(), "best_gradual_unfreeze_model_one.pth")
#         print(f"✅ Model disimpan (F1 terbaik baru: {best_acc:.4f})")



/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Menggunakan device: cuda
Ukuran gambar: 518x518, Batch Size: 8
Jumlah data training sebelum filtering manual: 4052

Mengidentifikasi kebocoran data (train vs test)...


Menghitung Hash Data Test:   0%|          | 0/2057 [00:00<?, ?it/s]

Menghitung Hash Data Train:   0%|          | 0/4052 [00:00<?, ?it/s]

Ditemukan 80 gambar di training set yang identik dengan gambar di test set.

Menerapkan strategi 'Validasi Bersih'...
Ukuran set training final (sisa bersih + semua bocor): 3257
Ukuran set validasi murni (hanya dari data bersih): 795



model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
# Impor tambahan yang Anda perlukan untuk evaluasi
from sklearn.metrics import accuracy_score, f1_score 
import timm # Pastikan timm sudah diimpor

###################################################################################
### BLOK B: MELATIH MODEL CONVNEXT ###
###################################################################################
print("\n" + "#"*80)
print("### MEMULAI BLOK B: TRAINING MODEL CONVNEXT ###") # <-- GANTI
print("#"*80 + "\n")

# B.1. Konfigurasi khusus untuk Model ConvNext
IMG_SIZE = 224
BATCH_SIZE = 16
BEST_MODEL_NAME_CONVNEXT = "best_convnext_standalone.pth" # <-- GANTI
print(f"Menggunakan Ukuran Gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# B.2. Buat Ulang Transforms dan DataLoaders untuk 224x224
# (Tidak ada perubahan, 224x224 sudah cocok untuk ConvNext)
train_transform_main_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
train_transform_extra_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([transforms.RandomRotation((90, 90)), transforms.RandomRotation((-90, -90))]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform_224 = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
train_dataset = DualTransformDataset(train_split, 'Filepath', 'Label', train_transform_main_224, train_transform_extra_224)
val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform_224)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

# B.3. Inisialisasi Model (DIGANTI KE CONVNEXT)
# <-- GANTI KELAS MODEL
class SingleConvNextModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # <-- GANTI ARSITEKTUR MODEL
        self.backbone = timm.create_model("convnext_base", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

model = SingleConvNextModel(num_classes=len(label_mapping)).to(DEVICE) # <-- GANTI
criterion = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.1)
scaler = torch.amp.GradScaler()
best_f1 = 0.0
best_acc = 0.0

# B.4. Lakukan Training (3 Tahap disesuaikan untuk ConvNext)
# --- TAHAP 1 ---
print("\n" + "="*50 + "\nTAHAP 1 (CONVNEXT): Melatih Classifier Head\n" + "="*50) # <-- GANTI
for param in model.backbone.parameters(): param.requires_grad = False
for param in model.backbone.head.parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1)
for epoch in range(EPOCHS_S1):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S1 ConvNext Epoch {epoch+1}/{EPOCHS_S1}") # <-- GANTI
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()

# --- TAHAP 2 ---
print("\n" + "="*50 + "\nTAHAP 2 (CONVNEXT): Melatih Head + Lapisan Atas\n" + "="*50) # <-- GANTI
# <-- GANTI LOGIKA UNFREEZING
# ConvNext menggunakan 'stages' bukan 'layers'
for param in model.backbone.stages[-1].parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2)
for epoch in range(EPOCHS_S2):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S2 ConvNext Epoch {epoch+1}/{EPOCHS_S2}") # <-- GANTI
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        
# --- TAHAP 3 ---
LR_S3_CONVNEXT = 1e-5 # <-- GANTI NAMA
print("\n" + "="*50 + "\nTAHAP 3 (CONVNEXT): Fine-tuning Seluruh Model\n" + "="*50) # <-- GANTI
for param in model.parameters(): param.requires_grad = True
optimizer = optim.AdamW(model.parameters(), lr=LR_S3_CONVNEXT, weight_decay=1e-4) # <-- GANTI LR
scheduler = scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS_S3, 
    eta_min=1e-6    
)

for epoch in range(EPOCHS_S3):
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"S3 ConvNext Epoch {epoch+1}/{EPOCHS_S3} (Train)") # <-- GANTI
    for images, labels in train_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer); scaler.update(); scheduler.step()
        train_loss += loss.item()
    model.eval()
    val_loss, all_preds, all_labels = 0.0, [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            with torch.amp.autocast(device_type = 'cuda'):
                outputs = model(images); loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy()); all_labels.extend(labels.cpu().numpy())
            
    f1 = f1_score(all_labels, all_preds, average="macro")
    acc = accuracy_score(all_labels, all_preds) 
    print(f"\nEpoch [{epoch+1}/{EPOCHS_S3}] | Val F1: {f1:.4f}| Val acc: {acc:.4f}")
    
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), BEST_MODEL_NAME_CONVNEXT) # <-- GANTI
        print(f"✅ Model ConvNext disimpan ke {BEST_MODEL_NAME_CONVNEXT} (acc terbaik baru: {best_acc:.4f})") # <-- GANTI

print(f"\nTraining Model ConvNext Selesai. Bobot terbaik disimpan di {BEST_MODEL_NAME_CONVNEXT}") # <-- GANTI

# B.5. Bersihkan Memori GPU
print("Membersihkan memori GPU sebelum inferensi...")
#del model, optimizer, scheduler, train_loader, val_loader, train_dataset, val_dataset, train_df, clean_df, leaked_df, train_split, val_split
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK B: TRAINING MODEL CONVNEXT ###
################################################################################

Menggunakan Ukuran Gambar: 224x224, Batch Size: 16


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]


TAHAP 1 (CONVNEXT): Melatih Classifier Head


S1 ConvNext Epoch 1/3:   0%|          | 0/408 [00:00<?, ?it/s]

S1 ConvNext Epoch 2/3:   0%|          | 0/408 [00:00<?, ?it/s]

...


Epoch [10/12] | Val F1: 0.9119| Val acc: 0.9119


S3 ConvNext Epoch 11/12 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [11/12] | Val F1: 0.9179| Val acc: 0.9182


S3 ConvNext Epoch 12/12 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [12/12] | Val F1: 0.9179| Val acc: 0.9182

Training Model ConvNext Selesai. Bobot terbaik disimpan di best_convnext_standalone.pth
Membersihkan memori GPU sebelum inferensi...


In [ ]:
###################################################################################
### BLOK C: INFERENSI DAN SUBMISI (CONVNEXT)
###################################################################################
print("\n" + "#"*80)
print("### MEMULAI BLOK C: INFERENSI DAN SUBMISI CONVNEXT ###")
print("#"*80 + "\n")
label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14
}
# C.1. Definisikan Custom Dataset untuk Data Tes
# (Menggunakan dataset tes yang bersih, hanya memuat gambar)
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        # Ambil semua nama file di direktori tes
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Mengembalikan gambar dan nama file (sebagai ID)
        return image, img_name

# C.2. Tentukan Transformasi untuk Data Tes
# (Kita gunakan transform validasi/tes dari Blok B)
test_transform = val_test_transform_224 
print(f"Menggunakan IMG_SIZE: {IMG_SIZE}x{IMG_SIZE}")

# C.3. Inisialisasi Dataset dan DataLoader
# Pastikan TEST_DIR sudah terdefinisi di skrip Anda
test_dataset = TestImageDataset(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# Buat reverse mapping: indeks ke nama kelas
idx_to_class = {v: k for k, v in label_mapping.items()}

# C.4. Inisialisasi Model dan Pemuatan Bobot
# Pastikan SingleConvNextModel() telah didefinisikan sebelumnya (di Blok B)
model_inferensi = SingleConvNextModel(num_classes=len(label_mapping)).to(DEVICE)
print(f"Memuat bobot dari: {BEST_MODEL_NAME_CONVNEXT}")

if os.path.exists(BEST_MODEL_NAME_CONVNEXT):
    model_inferensi.load_state_dict(torch.load(BEST_MODEL_NAME_CONVNEXT, map_location=DEVICE))
    print("Bobot ConvNext berhasil dimuat.")
else:
    print(f"❌ Error: File bobot {BEST_MODEL_NAME_CONVNEXT} tidak ditemukan!")
    # Hentikan proses jika bobot tidak ditemukan
    # raise FileNotFoundError(f"File bobot {BEST_MODEL_NAME_CONVNEXT} tidak ditemukan.")

# C.5. Melakukan Inferensi
model_inferensi.eval() # Set model ke mode evaluasi
image_ids = []
predictions = []

print("Memulai inferensi pada data tes...")
with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Inferensi ConvNext"):
        images = images.to(DEVICE)
        
        # Inferensi dengan autocast
        with torch.amp.autocast(device_type = 'cuda' if 'cuda' in DEVICE.type else 'cpu'):
            outputs = model_inferensi(images)
        
        # Ambil prediksi kelas
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        
        # Simpan nama file (ID) dan prediksi
        image_ids.extend(file_names)
        predictions.extend(preds)

print("Inferensi selesai. Total prediksi:", len(predictions))

# C.6. Pemetaan Ulang Label ke Nama Kelas
predicted_labels = [idx_to_class[p] for p in predictions]

# C.7. Membuat DataFrame Submisi
# Mengubah nama file (misal: "image123.jpg") menjadi ID (misal: "image123")
image_ids_no_ext = [os.path.splitext(f)[0] for f in image_ids]

submission_df = pd.DataFrame({
    'ID': image_ids_no_ext,
    'label': predicted_labels # Sesuaikan nama kolom 'label'/'style'
})

# C.8. Menyimpan ke File CSV
submission_filename = "submission_convnext.csv"
submission_df.to_csv(submission_filename, index=False)

print("\n" + "="*50)
print(f"🎉 Submisi berhasil dibuat!")
print(f"File submisi disimpan di: {submission_filename}")
print("Contoh 5 baris pertama:\n", submission_df.head())
print("="*50)

# C.9. Bersihkan Memori GPU 
print("Membersihkan memori GPU setelah inferensi...")
del model_inferensi, test_loader, test_dataset
torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: INFERENSI DAN SUBMISI CONVNEXT ###
################################################################################

Menggunakan IMG_SIZE: 224x224
Memuat bobot dari: best_convnext_standalone.pth
Bobot ConvNext berhasil dimuat.
Memulai inferensi pada data tes...


Inferensi ConvNext:   0%|          | 0/129 [00:00<?, ?it/s]

Inferensi selesai. Total prediksi: 2057

🎉 Submisi berhasil dibuat!
File submisi disimpan di: submission_convnext.csv
Contoh 5 baris pertama:
      ID        label
0  0001  Sate Padang
1  0002      Rendang
2  0003   Ayam Bakar
3  0004      Rendang
4  0005  Ayam Goreng
Membersihkan memori GPU setelah inferensi...


In [ ]:
###################################################################################
### BLOK B: MELATIH MODEL SWIN TRANSFORMER ###
###################################################################################
print("\n" + "#"*80)
print("### MEMULAI BLOK B: TRAINING MODEL SWIN TRANSFORMER ###")
print("#"*80 + "\n")

# B.1. Konfigurasi khusus untuk Model Swin
IMG_SIZE = 224
BATCH_SIZE = 16
BEST_MODEL_NAME_SWIN = "best_swin_standalone.pth"
print(f"Menggunakan Ukuran Gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# B.2. Buat Ulang Transforms dan DataLoaders untuk 224x224
train_transform_main_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
train_transform_extra_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([transforms.RandomRotation((90, 90)), transforms.RandomRotation((-90, -90))]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform_224 = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
train_dataset = DualTransformDataset(train_split, 'Filepath', 'Label', train_transform_main_224, train_transform_extra_224)
val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform_224)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

# B.3. Inisialisasi Model
model = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.1)
scaler = torch.amp.GradScaler()
best_f1 = 0.0
best_acc = 0.0
# B.4. Lakukan Training (3 Tahap disesuaikan untuk Swin)
# --- TAHAP 1 ---
print("\n" + "="*50 + "\nTAHAP 1 (SWIN): Melatih Classifier Head\n" + "="*50)
for param in model.backbone.parameters(): param.requires_grad = False
for param in model.backbone.head.parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1)
for epoch in range(EPOCHS_S1):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S1 Swin Epoch {epoch+1}/{EPOCHS_S1}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()

# --- TAHAP 2 ---
print("\n" + "="*50 + "\nTAHAP 2 (SWIN): Melatih Head + Lapisan Atas\n" + "="*50)
for param in model.backbone.layers[-1].parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2)
for epoch in range(EPOCHS_S2):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S2 Swin Epoch {epoch+1}/{EPOCHS_S2}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        
# --- TAHAP 3 ---
LR_S3_SWIN = 1e-5 
print("\n" + "="*50 + "\nTAHAP 3 (SWIN): Fine-tuning Seluruh Model\n" + "="*50)
for param in model.parameters(): param.requires_grad = True
optimizer = optim.AdamW(model.parameters(), lr=LR_S3_SWIN, weight_decay=1e-4)
scheduler = scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS_S3,   # jumlah iterasi (biasanya epoch)
    eta_min=1e-6       # nilai LR minimum saat akhir siklus
)

for epoch in range(EPOCHS_S3):
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"S3 Swin Epoch {epoch+1}/{EPOCHS_S3} (Train)")
    for images, labels in train_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer); scaler.update(); scheduler.step()
        train_loss += loss.item()
    model.eval()
    val_loss, all_preds, all_labels = 0.0, [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            with torch.amp.autocast(device_type = 'cuda'):
                outputs = model(images); loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy()); all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average="macro")
    acc = accuracy_score(all_labels, all_preds) 
    print(f"\nEpoch [{epoch+1}/{EPOCHS_S3}] | Val F1: {f1:.4f}| Val acc: {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), BEST_MODEL_NAME_SWIN)
        print(f"✅ Model Swin disimpan ke {BEST_MODEL_NAME_SWIN} (acc terbaik baru: {best_acc:.4f})")

print(f"\nTraining Model Swin Selesai. Bobot terbaik disimpan di {BEST_MODEL_NAME_SWIN}")
# B.5. Bersihkan Memori GPU
print("Membersihkan memori GPU sebelum inferensi...")
#del model, optimizer, scheduler, train_loader, val_loader, train_dataset, val_dataset, train_df, clean_df, leaked_df, train_split, val_split
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK B: TRAINING MODEL SWIN TRANSFORMER ###
################################################################################

Menggunakan Ukuran Gambar: 224x224, Batch Size: 16


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]


TAHAP 1 (SWIN): Melatih Classifier Head


S1 Swin Epoch 1/3:   0%|          | 0/408 [00:00<?, ?it/s]

S1 Swin Epoch 2/3:   0%|          | 0/408 [00:00<?, ?it/s]

...


Epoch [10/12] | Val F1: 0.9270| Val acc: 0.9270


S3 Swin Epoch 11/12 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [11/12] | Val F1: 0.9288| Val acc: 0.9283


S3 Swin Epoch 12/12 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [12/12] | Val F1: 0.9244| Val acc: 0.9245

Training Model Swin Selesai. Bobot terbaik disimpan di best_swin_standalone.pth
Membersihkan memori GPU sebelum inferensi...


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

# --- Asumsi Variabel Global Anda (Perlu Didefinisikan di Lingkungan Anda) ---
# Misalnya:
# test_dir = '/path/to/your/test_images'
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# IMG_SIZE = 224 # Dari Blok B.1
# BATCH_SIZE = 16 # Dari Blok B.1
BEST_MODEL_NAME_SWIN = "best_swin_standalone.pth" # Dari Blok B.1
# IMAGENET_MEAN = [0.485, 0.456, 0.406]
# IMAGENET_STD = [0.229, 0.224, 0.225]

# Asumsi definisi label_mapping Anda:
label_mapping = {
    "Ayam Bakar": 0, "Ayam Betutu": 1, "Ayam Goreng": 2, "Ayam Pop": 3,
    "Bakso": 4, "Coto Makassar": 5, "Gado Gado": 6, "Gudeg": 7,
    "Nasi Goreng": 8, "Pempek": 9, "Rawon": 10, "Rendang": 11,
    "Sate Madura": 12, "Sate Padang": 13, "Soto": 14
}

# Membuat inverse mapping: indeks ke nama kelas
idx_to_class = {v: k for k, v in label_mapping.items()}

print("\n" + "#"*80)
print("### MEMULAI BLOK C: INFERENSI DAN SUBMISI SWIN TRANSFORMER ###")
print("#"*80 + "\n")

# C.1. Definisikan Custom Dataset untuk Data Tes
# Data tes hanya memiliki gambar, tidak ada label.
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        # Ambil semua nama file di direktori tes
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Mengembalikan gambar dan nama file (sebagai ID)
        return image, img_name

# C.2. Tentukan Transformasi untuk Data Tes (Sama seperti val_test_transform_224)
test_transform = transforms.Compose([
    CLAHETransform(), # Pastikan CLAHETransform terdefinisi
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# C.3. Inisialisasi Dataset dan DataLoader
# Anda perlu mengganti '/path/to/your/test_dir' dengan lokasi direktori gambar tes Anda
# Contoh:
# test_dir = "test_data_folder" 
test_dataset = TestImageDataset(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) # num_workers 0 jika masalah, atau gunakan NUM_WORKERS

# C.4. Inisialisasi Model dan Pemuatan Bobot
# Pastikan SingleSwinModel() telah didefinisikan sebelumnya di skrip Anda
model_inferensi = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
print(f"Memuat bobot dari: {BEST_MODEL_NAME_SWIN}")

if os.path.exists(BEST_MODEL_NAME_SWIN):
    model_inferensi.load_state_dict(torch.load(BEST_MODEL_NAME_SWIN, map_location=DEVICE))
    print("Bobot Swin Transformer berhasil dimuat.")
else:
    print(f"❌ Error: File bobot {BEST_MODEL_NAME_SWIN} tidak ditemukan!")
    # Hentikan proses jika bobot tidak ditemukan
    # raise FileNotFoundError(f"File bobot {BEST_MODEL_NAME_SWIN} tidak ditemukan.")


# C.5. Melakukan Inferensi
model_inferensi.eval() # Set model ke mode evaluasi
image_ids = []
predictions = []

print("Memulai inferensi pada data tes...")
with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Inferensi"):
        images = images.to(DEVICE)
        
        # Inferensi dengan autocast (jika menggunakan GPU)
        with torch.amp.autocast(device_type = 'cuda' if 'cuda' in DEVICE.type else 'cpu'):
            outputs = model_inferensi(images)
        
        # Ambil prediksi kelas (indeks dengan probabilitas tertinggi)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        
        # Simpan nama file (ID) dan prediksi
        image_ids.extend(file_names)
        predictions.extend(preds)

print("Inferensi selesai. Total prediksi:", len(predictions))

# C.6. Pemetaan Ulang Label ke Nama Kelas
# Mengubah indeks numerik menjadi string nama kelas
predicted_labels = [idx_to_class[p] for p in predictions]

# C.7. Membuat DataFrame Submisi
submission_df = pd.DataFrame({
    'id': image_ids,
    'label': predicted_labels
})

# C.8. Menyimpan ke File CSV
submission_filename = "submission_swin_transformer.csv"
submission_df.to_csv(submission_filename, index=False)

print("\n" + "="*50)
print(f"🎉 Submisi berhasil dibuat!")
print(f"File submisi disimpan di: {submission_filename}")
print("Contoh 5 baris pertama:\n", submission_df.head())
print("="*50)

# C.9. Bersihkan Memori GPU (Opsional, sudah dilakukan di Blok B, tetapi baik untuk diulang)
# print("Membersihkan memori GPU setelah inferensi...")
# del model_inferensi
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: INFERENSI DAN SUBMISI SWIN TRANSFORMER ###
################################################################################

Memuat bobot dari: best_swin_standalone.pth
Bobot Swin Transformer berhasil dimuat.
Memulai inferensi pada data tes...


Inferensi:   0%|          | 0/129 [00:00<?, ?it/s]

Inferensi selesai. Total prediksi: 2057

🎉 Submisi berhasil dibuat!
File submisi disimpan di: submission_swin_transformer.csv
Contoh 5 baris pertama:
          id        label
0  0001.jpg  Sate Padang
1  0002.jpg  Nasi Goreng
2  0003.jpg  Ayam Goreng
3  0004.jpg      Rendang
4  0005.jpg  Ayam Goreng


In [ ]:
###################################################################################
### BLOK B: MELATIH MODEL EVA TRANSFORMER ###
###################################################################################
print("\n" + "#"*80)
print("### MEMULAI BLOK B: TRAINING MODEL EVA TRANSFORMER ###") # Ganti teks
print("#"*80 + "\n")

# B.1. Konfigurasi khusus untuk Model EVA
IMG_SIZE = 224
BATCH_SIZE = 16
# GANTI NAMA FILE BOBOT
BEST_MODEL_NAME_EVA = "best_eva_standalone.pth" 
print(f"Menggunakan Ukuran Gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# B.2. Buat Ulang Transforms dan DataLoaders untuk 224x224
# (Tidak ada perubahan di sini, kode Anda sudah benar)
train_transform_main_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
train_transform_extra_224 = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([transforms.RandomRotation((90, 90)), transforms.RandomRotation((-90, -90))]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform_224 = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
train_dataset = DualTransformDataset(train_split, 'Filepath', 'Label', train_transform_main_224, train_transform_extra_224)
val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform_224)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)


# B.3. Inisialisasi Model (DIGANTI KE EVA)
# Kita ganti nama kelasnya agar jelas
class SingleEvaModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # GANTI MODEL DI SINI: dari "swin_base..." ke "eva02_base..."
        self.backbone = timm.create_model("eva02_base_patch14_224", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

model = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE) # Panggil kelas baru
print(f"Menggunakan model: eva02_base_patch14_224")

criterion = nn.CrossEntropyLoss(weight=class_weights,label_smoothing=0.1)
scaler = torch.amp.GradScaler()
best_f1 = 0.0
best_acc = 0.0

# B.4. Lakukan Training (3 Tahap disesuaikan untuk EVA)
# --- TAHAP 1 ---
print("\n" + "="*50 + "\nTAHAP 1 (EVA): Melatih Classifier Head\n" + "="*50)
for param in model.backbone.parameters(): param.requires_grad = False
# Model EVA (dan ViT) biasanya juga menggunakan 'head'
for param in model.backbone.head.parameters(): param.requires_grad = True 
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1)
for epoch in range(EPOCHS_S1):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S1 EVA Epoch {epoch+1}/{EPOCHS_S1}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()

# --- TAHAP 2 (LOGIKA UNFREEZE BERBEDA) ---
print("\n" + "="*50 + "\nTAHAP 2 (EVA): Melatih Head + Blok Atas\n" + "="*50)
# Model ViT/EVA menggunakan 'blocks', bukan 'layers'
# Kita unfreeze blok terakhir
for param in model.backbone.blocks[-1].parameters(): param.requires_grad = True
# Juga unfreeze LayerNorm terakhir sebelum head
for param in model.backbone.norm.parameters(): param.requires_grad = True
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2)
for epoch in range(EPOCHS_S2):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"S2 EVA Epoch {epoch+1}/{EPOCHS_S2}")
    for images, labels in progress_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        
# --- TAHAP 3 ---

LR_S3_EVA = 1e-5 # Anda bisa menggunakan LR yang sama atau berbeda
print("\n" + "="*50 + "\nTAHAP 3 (EVA): Fine-tuning Seluruh Model\n" + "="*50)
for param in model.parameters(): param.requires_grad = True
optimizer = optim.AdamW(model.parameters(), lr=LR_S3_EVA, weight_decay=1e-4)
scheduler = scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS_S3,   # jumlah iterasi (biasanya epoch)
    eta_min=1e-6       # nilai LR minimum saat akhir siklus
)

for epoch in range(EPOCHS_S3):
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"S3 EVA Epoch {epoch+1}/{EPOCHS_S3} (Train)")
    for images, labels in train_bar:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        with torch.amp.autocast(device_type = 'cuda'):
            outputs = model(images); loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer); scaler.update(); scheduler.step()
        train_loss += loss.item()
    model.eval()
    val_loss, all_preds, all_labels = 0.0, [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            with torch.amp.autocast(device_type = 'cuda'):
                outputs = model(images); loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy()); all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average="macro")
    acc = accuracy_score(all_labels, all_preds) 
    print(f"\nEpoch [{epoch+1}/{EPOCHS_S3}] | Val F1: {f1:.4f}| Val acc: {acc:.4f}")
    if acc > best_acc:
        best_acc = acc
        # GANTI NAMA FILE DISINI
        torch.save(model.state_dict(), BEST_MODEL_NAME_EVA) 
        print(f"✅ Model EVA disimpan ke {BEST_MODEL_NAME_EVA} (F1 terbaik baru: {best_acc:.4f})") # Ganti teks

print(f"\nTraining Model EVA Selesai. Bobot terbaik disimpan di {BEST_MODEL_NAME_EVA}") # Ganti teks

# B.5. Bersihkan Memori GPU
# print("Membersihkan memori GPU sebelum inferensi...")
# del model, optimizer, scheduler, train_loader, val_loader, train_dataset, val_dataset, train_df, clean_df, leaked_df, train_split, val_split
# torch.cuda.empty_cache()
# (Asumsi impor dan variabel global sudah ada dari kode sebelumnya)

# Pastikan Anda MENDEFINISIKAN ULANG kelas SingleEvaModel di sini
# agar skrip inferensi tahu strukturnya.
class SingleEvaModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # Pastikan nama model sama persis dengan saat training
        self.backbone = timm.create_model("eva02_base_patch14_224", pretrained=False, num_classes=num_classes) # pretrained=False saat inferensi
    def forward(self, x):
        return self.backbone(x)

# GANTI NAMA FILE BOBOT
BEST_MODEL_NAME_EVA = "best_eva_standalone.pth"
# (Asumsi idx_to_class, TEST_DIR, test_transform, dll sudah terdefinisi)
# ...

print("\n" + "#"*80)
print("### MEMULAI BLOK C: INFERENSI DAN SUBMISI EVA TRANSFORMER ###") # Ganti teks
print("#"*80 + "\n")

# C.1. Definisikan Custom Dataset untuk Data Tes
# (Tidak ada perubahan di sini, TestImageDataset sudah benar)
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.transform = transform
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# C.2. Tentukan Transformasi untuk Data Tes
# (Tidak ada perubahan di sini, test_transform sudah benar)
test_transform = transforms.Compose([
    CLAHETransform(), 
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# C.3. Inisialisasi Dataset dan DataLoader
# (Tidak ada perubahan di sini)
test_dataset = TestImageDataset(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) 

# C.4. Inisialisasi Model dan Pemuatan Bobot (DIGANTI KE EVA)
model_inferensi = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE) # Panggil kelas EVA
print(f"Memuat bobot dari: {BEST_MODEL_NAME_EVA}") # Ganti variabel

if os.path.exists(BEST_MODEL_NAME_EVA): # Ganti variabel
    model_inferensi.load_state_dict(torch.load(BEST_MODEL_NAME_EVA, map_location=DEVICE)) # Ganti variabel
    print("Bobot EVA Transformer berhasil dimuat.") # Ganti teks
else:
    print(f"❌ Error: File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan!") # Ganti variabel
    # raise FileNotFoundError(f"File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan.")


# C.5. Melakukan Inferensi
# (Tidak ada perubahan di sini)
model_inferensi.eval() 
image_ids = []
predictions = []

print("Memulai inferensi pada data tes...")
with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Inferensi"):
        images = images.to(DEVICE)
        with torch.amp.autocast(device_type = 'cuda' if 'cuda' in DEVICE.type else 'cpu'):
            outputs = model_inferensi(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        image_ids.extend(file_names)
        predictions.extend(preds)

print("Inferensi selesai. Total prediksi:", len(predictions))

# C.6. Pemetaan Ulang Label ke Nama Kelas
# (Tidak ada perubahan di sini)
predicted_labels = [idx_to_class[p] for p in predictions]

# C.7. Membuat DataFrame Submisi
# (Tidak ada perubahan di sini)
submission_df = pd.DataFrame({
    'id': image_ids,
    'label': predicted_labels
})

# C.8. Menyimpan ke File CSV (GANTI NAMA FILE)
submission_filename = "submission_eva_transformer.csv" 
submission_df.to_csv(submission_filename, index=False)

print("\n" + "="*50)
print(f"🎉 Submisi berhasil dibuat!")
print(f"File submisi disimpan di: {submission_filename}")
print("Contoh 5 baris pertama:\n", submission_df.head())
print("="*50)

# C.9. Bersihkan Memori GPU
print("Membersihkan memori GPU setelah inferensi...")
# del model_inferensi
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK B: TRAINING MODEL EVA TRANSFORMER ###
################################################################################

Menggunakan Ukuran Gambar: 224x224, Batch Size: 16


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Menggunakan model: eva02_base_patch14_224

TAHAP 1 (EVA): Melatih Classifier Head


S1 EVA Epoch 1/3:   0%|          | 0/408 [00:00<?, ?it/s]

S1 EVA Epoch 2/3:   0%|          | 0/408 [00:00<?, ?it/s]

...


Epoch [11/12] | Val F1: 0.9364| Val acc: 0.9371


S3 EVA Epoch 12/12 (Train):   0%|          | 0/408 [00:00<?, ?it/s]


Epoch [12/12] | Val F1: 0.9364| Val acc: 0.9371

Training Model EVA Selesai. Bobot terbaik disimpan di best_eva_standalone.pth

################################################################################
### MEMULAI BLOK C: INFERENSI DAN SUBMISI EVA TRANSFORMER ###
################################################################################

Memuat bobot dari: best_eva_standalone.pth
Bobot EVA Transformer berhasil dimuat.
Memulai inferensi pada data tes...


Inferensi:   0%|          | 0/129 [00:00<?, ?it/s]

Inferensi selesai. Total prediksi: 2057

🎉 Submisi berhasil dibuat!
File submisi disimpan di: submission_eva_transformer.csv
Contoh 5 baris pertama:
          id        label
0  0001.jpg  Sate Padang
1  0002.jpg      Rendang
2  0003.jpg  Ayam Betutu
3  0004.jpg      Rendang
4  0005.jpg  Ayam Goreng
Membersihkan memori GPU setelah inferensi...


In [ ]:
# C.3. Inisialisasi Dataset dan DataLoader
# (Tidak ada perubahan di sini)
test_dataset = TestImageDataset(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) 

# C.4. Inisialisasi Model dan Pemuatan Bobot (DIGANTI KE EVA)
model_inferensi = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE) # Panggil kelas EVA
print(f"Memuat bobot dari: {BEST_MODEL_NAME_EVA}") # Ganti variabel

if os.path.exists(BEST_MODEL_NAME_EVA): # Ganti variabel
    model_inferensi.load_state_dict(torch.load(BEST_MODEL_NAME_EVA, map_location=DEVICE)) # Ganti variabel
    print("Bobot EVA Transformer berhasil dimuat.") # Ganti teks
else:
    print(f"❌ Error: File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan!") # Ganti variabel
    # raise FileNotFoundError(f"File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan.")

label_mapping = {
    "Ayam Bakar": 0, "Ayam Betutu": 1, "Ayam Goreng": 2, "Ayam Pop": 3,
    "Bakso": 4, "Coto Makassar": 5, "Gado Gado": 6, "Gudeg": 7,
    "Nasi Goreng": 8, "Pempek": 9, "Rawon": 10, "Rendang": 11,
    "Sate Madura": 12, "Sate Padang": 13, "Soto": 14
}

# Membuat inverse mapping: indeks ke nama kelas
idx_to_class = {v: k for k, v in label_mapping.items()}
# C.5. Melakukan Inferensi
# (Tidak ada perubahan di sini)
model_inferensi.eval() 
image_ids = []
predictions = []

print("Memulai inferensi pada data tes...")
with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Inferensi"):
        images = images.to(DEVICE)
        with torch.amp.autocast(device_type = 'cuda' if 'cuda' in DEVICE.type else 'cpu'):
            outputs = model_inferensi(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        image_ids.extend(file_names)
        predictions.extend(preds)

print("Inferensi selesai. Total prediksi:", len(predictions))

# C.6. Pemetaan Ulang Label ke Nama Kelas
# (Tidak ada perubahan di sini)
predicted_labels = [idx_to_class[p] for p in predictions]

# C.7. Membuat DataFrame Submisi
# (Tidak ada perubahan di sini)
submission_df = pd.DataFrame({
    'id': image_ids,
    'label': predicted_labels
})

# C.8. Menyimpan ke File CSV (GANTI NAMA FILE)
submission_filename = "submission_eva_transformer.csv" 
submission_df.to_csv(submission_filename, index=False)

print("\n" + "="*50)
print(f"🎉 Submisi berhasil dibuat!")
print(f"File submisi disimpan di: {submission_filename}")
print("Contoh 5 baris pertama:\n", submission_df.head())
print("="*50)

# C.9. Bersihkan Memori GPU
print("Membersihkan memori GPU setelah inferensi...")
# del model_inferensi
# torch.cuda.empty_cache()

Memuat bobot dari: best_eva_standalone.pth
Bobot EVA Transformer berhasil dimuat.
Memulai inferensi pada data tes...


Inferensi:   0%|          | 0/129 [00:00<?, ?it/s]

KeyboardInterrupt: 

# BATAS

In [ ]:
# ===================================================================
# === 3. FUNGSI DAN KELAS HELPER (TIDAK DIUBAH) ===
# ===================================================================
def convert_path_to_df(dataset, is_test=False):
    image_dir = Path(dataset)
    filepaths = list(image_dir.glob(r'**/*.*'))
    if is_test:
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        return pd.DataFrame({'Filepath': filepaths})
    else:
        labels = [p.parts[-2] for p in filepaths]
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        labels = pd.Series(labels, name='Label')
        return pd.concat([filepaths, labels], axis=1)

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column=None, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform: image = self.transform(image)
        if self.label_column:
            label = self.dataframe.iloc[idx][self.label_column]
            return image, torch.tensor(label, dtype=torch.long)
        return image
class DualTransformDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column, transform_main, transform_extra):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform_main = transform_main
        self.transform_extra = transform_extra

    def __len__(self):
        # Gandakan ukuran dataset (dua versi per gambar)
        return len(self.dataframe) * 2

    def __getitem__(self, idx):
        # Tentukan apakah pakai augmentasi utama atau tambahan
        base_idx = idx // 2
        use_extra = idx % 2 == 1

        row = self.dataframe.iloc[base_idx]
        img_path = row[self.image_column]
        label = torch.tensor(row[self.label_column], dtype=torch.long)

        image = Image.open(img_path).convert('RGB')

        if use_extra:
            image = self.transform_extra(image)
        else:
            image = self.transform_main(image)

        return image, label

class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    def __call__(self, img):
        img_np = np.array(img); img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2Lab)
        l, a, b = cv2.split(img_lab); l_clahe = self.clahe.apply(l)
        img_lab_clahe = cv2.merge((l_clahe, a, b))
        img_rgb_clahe = cv2.cvtColor(img_lab_clahe, cv2.COLOR_Lab2RGB)
        return Image.fromarray(img_rgb_clahe)

class TestDataset(Dataset):
    def __init__(self, dataframe, image_column, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# ===================================================================
# === 4. PERSIAPAN DATA DENGAN STRATEGI "VALIDASI BERSIH" (DIUBAH) ===
# ===================================================================
def get_phash(filepath):
    try:
        with Image.open(filepath) as img: return imagehash.phash(img)
    except Exception: return None

# --- Langkah 4.1: Memuat dan memfilter data training secara manual ---
train_df = convert_path_to_df(TRAIN_DIR)
print(f"Jumlah data training sebelum filtering manual: {len(train_df)}")

# --- Langkah 4.2: Identifikasi Kebocoran dan Pisahkan Data ---
test_df = convert_path_to_df(TEST_DIR, is_test=True)
print("\n" + "="*50)
print("Mengidentifikasi kebocoran data (train vs test)...")
tqdm.pandas(desc="Menghitung Hash Data Test")
test_hashes = set(test_df['Filepath'].progress_apply(get_phash))
test_hashes.discard(None)

tqdm.pandas(desc="Menghitung Hash Data Train")
train_df['hash'] = train_df['Filepath'].progress_apply(get_phash)

train_df['is_leak'] = train_df['hash'].isin(test_hashes)
print(f"Ditemukan {train_df['is_leak'].sum()} gambar di training set yang identik dengan gambar di test set.")

print("\nMenerapkan strategi 'Validasi Bersih'...")
leaked_df = train_df[train_df['is_leak']].copy()
clean_df = train_df[~train_df['is_leak']].copy()

# --- Langkah 4.3: Pemetaan Label dan Split Data ---
label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14
}
clean_df['Label'] = clean_df['Label'].map(label_mapping)
leaked_df['Label'] = leaked_df['Label'].map(label_mapping)

val_split = pd.DataFrame()
if not clean_df.empty:
    try:
        clean_train_split, val_split = train_test_split(
            clean_df, test_size=0.2, random_state=42, stratify=clean_df['Label'])
    except ValueError:
        print("Peringatan: Gagal stratify, menggunakan split biasa.")
        clean_train_split, val_split = train_test_split(clean_df, test_size=0.2, random_state=42)
else:
    clean_train_split = clean_df

Jumlah data training sebelum filtering manual: 4065

Mengidentifikasi kebocoran data (train vs test)...


Menghitung Hash Data Test:   0%|          | 0/2057 [00:00<?, ?it/s]

Menghitung Hash Data Train:   0%|          | 0/4065 [00:00<?, ?it/s]

Ditemukan 80 gambar di training set yang identik dengan gambar di test set.

Menerapkan strategi 'Validasi Bersih'...


In [ ]:
# import torch.nn.functional as F

# ###################################################################################
# ### BLOK C: FINAL ENSEMBLE INFERENCE & SUBMISSION ###
# ###################################################################################
# print("\n" + "#"*80)
# print("### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION ###")
# print("#"*80 + "\n")

# # C.1. Inisialisasi kedua model untuk inferensi
# model_a = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
# model_b = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)

# # C.2. Muat bobot yang baru saja dilatih
# print(f"Memuat Model A dari: best_gradual_unfreeze_model")
# model_a.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/1/best_gradual_unfreeze_model_one (1).pth"))
# print(f"Memuat Model B dari: best_swin_standalone.pth")
# model_b.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/1/best_swin_standalone (1).pth"))
# model_a.eval()
# model_b.eval()

# # C.3. Siapkan Test Loader (resolusi tinggi untuk input awal)
# final_test_transform = transforms.Compose([
#     CLAHETransform(),
#     transforms.Resize((518, 518)), # Gunakan IMG_SIZE untuk Fusion
#     transforms.ToTensor(),
#     transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
# ])
# test_df = convert_path_to_df(TEST_DIR, is_test=True)
# test_dataset = TestDataset(test_df, 'Filepath', transform=final_test_transform)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
# reverse_label_mapping = {v: k for k, v in label_mapping.items()}
# submission_data = []

# # C.4. Lakukan prediksi dan ensemble voting
# with torch.no_grad():
#     for images, paths in tqdm(test_loader, desc="Membuat Prediksi Ensemble Final"):
#         images_518 = images.to(DEVICE)
#         outputs_a = model_a(images_518)
#         images_224 = F.interpolate(images_518, size=(224, 224), mode='bilinear', align_corners=False)
#         outputs_b = model_b(images_224)
#         probs_a = torch.softmax(outputs_a, dim=1)
#         probs_b = torch.softmax(outputs_b, dim=1)
#         avg_probs = (probs_a + probs_b) / 2.0
#         preds = torch.argmax(avg_probs, dim=1)
#         for i, path in enumerate(paths):
#             img_id = os.path.splitext(os.path.basename(path))[0]
#             label_int = preds[i].item()
#             label_str = reverse_label_mapping[label_int]
#             submission_data.append({'id': img_id, 'style': label_str})

# # C.5. Simpan hasil akhir
# print("Menyimpan hasil prediksi ke submission_ensemble_final.csv...")
# submission_df = pd.DataFrame(submission_data)
# submission_df.sort_values(by='id', inplace=True)
# submission_df.to_csv("submission_ensemble_final.csv", index=False)
# print("✅ File submission_ensemble_final.csv berhasil dibuat!")
# print("\n" + "="*50 + "\nSELURUH PROSES SELESAI\n" + "="*50)

In [ ]:
###################################################################################
### BLOK C: FINAL ENSEMBLE INFERENCE & SUBMISSION (MULTI-MODEL) ###
###################################################################################
import torch.nn.functional as F

print("\n" + "#"*80)
print("### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (MULTI-MODEL) ###")
print("#"*80 + "\n")

# C.1. Inisialisasi ketiga model untuk inferensi
model_a = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)   # Fusion
model_b = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE) # Swin
model_c = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE)  # EVA

# C.2. Muat bobot yang baru saja dilatih
print(f"Memuat Model A (FusionDINOv2) dari: best_gradual_unfreeze_model.pth")
model_a.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/1/best_gradual_unfreeze_model_one (1).pth", map_location=DEVICE))
print(f"Memuat Model B (SingleSwinModel) dari: best_swin_standalone.pth")
model_b.load_state_dict(torch.load("/kaggle/working/best_swin_standalone.pth.pth", map_location=DEVICE))
print(f"Memuat Model C (SingleEVAModel) dari: EVA.pth")
model_c.load_state_dict(torch.load("/kaggle/working/best_eva_standalone.pth", map_location=DEVICE))

# Set ke eval mode
model_a.eval()
model_b.eval()
model_c.eval()

# C.3. Siapkan Test Loader (resolusi tinggi untuk input awal)
final_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((518, 518)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
test_df = convert_path_to_df(TEST_DIR, is_test=True)
test_dataset = TestDataset(test_df, 'Filepath', transform=final_test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# C.4. Fungsi bantu untuk ensemble
def make_submission(ensemble_name, prob_list, paths):
    avg_probs = torch.stack(prob_list).mean(dim=0)
    preds = torch.argmax(avg_probs, dim=1)
    submission_data = []
    for i, path in enumerate(paths):
        img_id = os.path.splitext(os.path.basename(path))[0]
        label_int = preds[i].item()
        label_str = reverse_label_mapping[label_int]
        submission_data.append({'ID': img_id, 'label': label_str})
    df = pd.DataFrame(submission_data)
    df.sort_values(by='ID', inplace=True)
    filename = f"submission_{ensemble_name}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ File {filename} berhasil dibuat!")

# C.5. Proses inference
print("\nMembuat prediksi ensemble dari ketiga model...")
all_probs_a, all_probs_b, all_probs_c, all_paths = [], [], [], []

with torch.no_grad():
    for images, paths in tqdm(test_loader, desc="Inferensi Multi-Model"):
        images_518 = images.to(DEVICE)

        # Model FusionDINOv2 dan EVA pakai resolusi 518
        outputs_a = model_a(images_518)
        

        # Model Swin pakai resolusi 224
        images_224 = F.interpolate(images_518, size=(224, 224), mode='bilinear', align_corners=False)
        outputs_b = model_b(images_224)
        outputs_c = model_c(images_224)

        # Softmax probabilities
        probs_a = torch.softmax(outputs_a, dim=1)
        probs_b = torch.softmax(outputs_b, dim=1)
        probs_c = torch.softmax(outputs_c, dim=1)

        # Simpan hasil batch
        all_probs_a.append(probs_a.cpu())
        all_probs_b.append(probs_b.cpu())
        all_probs_c.append(probs_c.cpu())
        all_paths.extend(paths)

# Gabungkan semua batch
all_probs_a = torch.cat(all_probs_a, dim=0)
all_probs_b = torch.cat(all_probs_b, dim=0)
all_probs_c = torch.cat(all_probs_c, dim=0)

# C.6. Buat berbagai kombinasi ensemble
print("\nMenyusun berbagai kombinasi ensemble...")

# 1. EVA + SWIN
make_submission("ensemble_eva_swin", [all_probs_c, all_probs_b], all_paths)

# 2. EVA + FUSION
make_submission("ensemble_eva_fusion", [all_probs_c, all_probs_a], all_paths)

# 3. SWIN + FUSION
make_submission("ensemble_swin_fusion", [all_probs_b, all_probs_a], all_paths)

# 4. EVA + SWIN + FUSION
make_submission("ensemble_eva_swin_fusion", [all_probs_c, all_probs_b, all_probs_a], all_paths)

print("\n" + "="*50 + "\nSELURUH PROSES ENSEMBLE SELESAI\n" + "="*50)


In [ ]:
# ===================================================================
# === 2. KONFIGURASI DAN SETUP (TIDAK DIUBAH) ===
# ===================================================================
# Direktori
TRAIN_DIR = "/kaggle/input/newdatasets/train"
TEST_DIR = "/kaggle/input/newdatasets/test"

# Konfigurasi Model dan Training
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 518
BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count()

# --- HYPERPARAMETER UNTUK GRADUAL UNFREEZING ---
EPOCHS_S1 = 3
LR_S1 = 3e-4
EPOCHS_S2 = 5
LR_S2 = 5e-5
EPOCHS_S3 = 12
LR_S3 = 1.59e-06

# Statistik ImageNet
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

print(f"Menggunakan device: {DEVICE}")
print(f"Ukuran gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

# ===================================================================
# === 3. FUNGSI DAN KELAS HELPER (TIDAK DIUBAH) ===
# ===================================================================
def convert_path_to_df(dataset, is_test=False):
    image_dir = Path(dataset)
    filepaths = list(image_dir.glob(r'**/*.*'))
    if is_test:
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        return pd.DataFrame({'Filepath': filepaths})
    else:
        labels = [p.parts[-2] for p in filepaths]
        filepaths = pd.Series(filepaths, name='Filepath').astype(str)
        labels = pd.Series(labels, name='Label')
        return pd.concat([filepaths, labels], axis=1)

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column=None, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform
    def __len__(self): return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform: image = self.transform(image)
        if self.label_column:
            label = self.dataframe.iloc[idx][self.label_column]
            return image, torch.tensor(label, dtype=torch.long)
        return image
class DualTransformDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column, transform_main, transform_extra):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform_main = transform_main
        self.transform_extra = transform_extra

    def __len__(self):
        # Gandakan ukuran dataset (dua versi per gambar)
        return len(self.dataframe) * 2

    def __getitem__(self, idx):
        # Tentukan apakah pakai augmentasi utama atau tambahan
        base_idx = idx // 2
        use_extra = idx % 2 == 1

        row = self.dataframe.iloc[base_idx]
        img_path = row[self.image_column]
        label = torch.tensor(row[self.label_column], dtype=torch.long)

        image = Image.open(img_path).convert('RGB')

        if use_extra:
            image = self.transform_extra(image)
        else:
            image = self.transform_main(image)

        return image, label

class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    def __call__(self, img):
        img_np = np.array(img); img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2Lab)
        l, a, b = cv2.split(img_lab); l_clahe = self.clahe.apply(l)
        img_lab_clahe = cv2.merge((l_clahe, a, b))
        img_rgb_clahe = cv2.cvtColor(img_lab_clahe, cv2.COLOR_Lab2RGB)
        return Image.fromarray(img_rgb_clahe)

class TestDataset(Dataset):
    def __init__(self, dataframe, image_column, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.transform = transform
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_path

# ===================================================================
# === 4. PERSIAPAN DATA DENGAN STRATEGI "VALIDASI BERSIH" (DIUBAH) ===
# ===================================================================
def get_phash(filepath):
    try:
        with Image.open(filepath) as img: return imagehash.phash(img)
    except Exception: return None

# --- Langkah 4.1: Memuat dan memfilter data training secara manual ---
train_df = convert_path_to_df(TRAIN_DIR)
print(f"Jumlah data training sebelum filtering manual: {len(train_df)}")

# --- Langkah 4.2: Identifikasi Kebocoran dan Pisahkan Data ---
test_df = convert_path_to_df(TEST_DIR, is_test=True)
print("\n" + "="*50)
print("Mengidentifikasi kebocoran data (train vs test)...")
tqdm.pandas(desc="Menghitung Hash Data Test")
test_hashes = set(test_df['Filepath'].progress_apply(get_phash))
test_hashes.discard(None)

tqdm.pandas(desc="Menghitung Hash Data Train")
train_df['hash'] = train_df['Filepath'].progress_apply(get_phash)

train_df['is_leak'] = train_df['hash'].isin(test_hashes)
print(f"Ditemukan {train_df['is_leak'].sum()} gambar di training set yang identik dengan gambar di test set.")

print("\nMenerapkan strategi 'Validasi Bersih'...")
leaked_df = train_df[train_df['is_leak']].copy()
clean_df = train_df[~train_df['is_leak']].copy()

# --- Langkah 4.3: Pemetaan Label dan Split Data ---
label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14
}
clean_df['Label'] = clean_df['Label'].map(label_mapping)
leaked_df['Label'] = leaked_df['Label'].map(label_mapping)

val_split = pd.DataFrame()
if not clean_df.empty:
    try:
        clean_train_split, val_split = train_test_split(
            clean_df, test_size=0.2, random_state=42, stratify=clean_df['Label'])
    except ValueError:
        print("Peringatan: Gagal stratify, menggunakan split biasa.")
        clean_train_split, val_split = train_test_split(clean_df, test_size=0.2, random_state=42)
else:
    clean_train_split = clean_df

train_split = pd.concat([clean_train_split, leaked_df], ignore_index=True)
train_split.drop(columns=['hash', 'is_leak'], inplace=True)
val_split.drop(columns=['hash', 'is_leak'], inplace=True)
print(f"Ukuran set training final (sisa bersih + semua bocor): {len(train_split)}")
print(f"Ukuran set validasi murni (hanya dari data bersih): {len(val_split)}")
print("="*50 + "\n")

# --- Langkah 4.4: Lanjutkan dengan pipeline seperti biasa ---
train_transform = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN,std=IMAGENET_STD)
])
# Transform tambahan: rotasi 90° kiri/kanan
train_transform_extra = transforms.Compose([
    CLAHETransform(),
    transforms.RandomChoice([
        transforms.RandomRotation((90, 90)),
        transforms.RandomRotation((-90, -90))
    ]),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

train_dataset = DualTransformDataset(
    train_split,
    image_column='Filepath',
    label_column='Label',
    transform_main=train_transform,
    transform_extra=train_transform_extra
)

val_dataset = CustomDataset(val_split, 'Filepath', 'Label', val_test_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)
class_weights = compute_class_weight("balanced", classes=np.unique(train_split["Label"]), y=train_split["Label"])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)


# ===================================================================
# === 5. DEFINISI MODEL (TIDAK DIUBAH) ===
# ===================================================================
class FusionDINOv2(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.dinov2 = timm.create_model("vit_base_patch14_dinov2", pretrained=True, num_classes=0)
        self.convnext = timm.create_model("convnext_tiny", pretrained=True, num_classes=0)
        fusion_dim = self.dinov2.num_features + self.convnext.num_features
        self.classifier = nn.Sequential(
            nn.Linear(fusion_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, x):
        feat_dino = self.dinov2(x)
        feat_conv = self.convnext(x)
        feat_combined = torch.cat((feat_dino, feat_conv), dim=1)
        out = self.classifier(feat_combined)
        return out
class SingleSwinModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.backbone = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)
class SingleEVAModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # Menggunakan model EVA-02 yang sangat kuat
        model_name = "eva02_base_patch14_224.mim_in22k"
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

Menggunakan device: cuda
Ukuran gambar: 518x518, Batch Size: 8
Jumlah data training sebelum filtering manual: 4065

Mengidentifikasi kebocoran data (train vs test)...


Menghitung Hash Data Test:   0%|          | 0/2057 [00:00<?, ?it/s]

Menghitung Hash Data Train:   0%|          | 0/4065 [00:00<?, ?it/s]

Ditemukan 80 gambar di training set yang identik dengan gambar di test set.

Menerapkan strategi 'Validasi Bersih'...
Ukuran set training final (sisa bersih + semua bocor): 3268
Ukuran set validasi murni (hanya dari data bersih): 797



In [ ]:
class SingleEVAModel(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        # Menggunakan model EVA-02 yang sangat kuat
        model_name = "eva02_base_patch14_224.mim_in22k"
        self.backbone = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    def forward(self, x):
        return self.backbone(x)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ttach as tta  # <-- Impor TTA
import pandas as pd
import os
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

# =================================================================================
# ASUMSI: Variabel dan Kelas ini sudah ada dari blok sebelumnya:
#
# - DEVICE = torch.device(...)
# - label_mapping = {...}
# - TEST_DIR = "..."
# - BATCH_SIZE, NUM_WORKERS, IMAGENET_MEAN, IMAGENET_STD
# - Class Definition: FusionDINOv2, SingleSwinModel, SingleEVAModel
# - Class Definition: CLAHETransform, TestDataset
# - Fungsi Definition: convert_path_to_df
# =================================================================================


####################################################################################
### BLOK C: FINAL ENSEMBLE INFERENCE & SUBMISSION (DENGAN TTA)
####################################################################################

print("\n" + "#"*80)
print("### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (MULTI-MODEL TTA) ###")
print("#"*80 + "\n")

# C.1. Definisikan TTA transformations
# Kita akan menggunakan Horizontal Flip dan 4 Rotasi (0, 90, 180, 270)
# Total augmentasi = 2 * 4 = 8 augmentasi per gambar
print("Mendefinisikan TTA transforms (Flip + Rotasi)...")
tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 90, 180, 270]), 
        # tta.Scale(scales=[1, 1.1], use_fast_collate=True), # <-- Bisa ditambah jika perlu
    ]
)

# C.2. Buat Wrapper Model untuk menangani resizing
# Ini adalah cara yang "benar" agar TTA (misal: rotasi)
# diterapkan pada gambar 518px SEBELUM di-resize ke 224px.
class Model224Wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        print(" > Wrapper 224px dibuat")

    def forward(self, x_518):
        # TTA Wrapper akan memberikan gambar 518px yang sudah di-augmentasi
        # Kita resize ke 224px DI DALAM forward pass
        x_224 = F.interpolate(x_518, size=(224, 224), mode='bilinear', align_corners=False)
        return self.model(x_224)

# C.3. Inisialisasi model ASLI (tanpa TTA)
print("\nInisialisasi model dasar...")
model_a_base = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
model_b_base = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
model_c_base = SingleEVAModel(num_classes=len(label_mapping)).to(DEVICE)

# C.4. Muat bobot ke model ASLI
print("Memuat bobot...")
print(f"Memuat Model A (FusionDINOv2) dari: best_gradual_unfreeze_model_one.pth")
model_a_base.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/4/best_gradual_unfreeze_model_one (1).pth", map_location=DEVICE))
print(f"Memuat Model B (SingleSwinModel) dari: best_swin_standalone.pth")
model_b_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_swin_standalone (3).pth", map_location=DEVICE))
print(f"Memuat Model C (SingleEVAModel) dari: EVA.pth")
model_c_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_eva_standalone (3).pth", map_location=DEVICE))

# C.5. BUNGKUS model dengan TTA
print("\nMembungkus model dengan TTA Wrapper...")
# Model A (Fusion/DINO) menggunakan 518px, bisa langsung dibungkus
tta_model_a = tta.ClassificationTTAWrapper(model_a_base, tta_transforms).to(DEVICE)

# Model B (Swin) dan C (EVA) dibungkus DENGAN wrapper 224px kita
tta_model_b = tta.ClassificationTTAWrapper(
    Model224Wrapper(model_b_base), tta_transforms
).to(DEVICE)

tta_model_c = tta.ClassificationTTAWrapper(
    Model224Wrapper(model_c_base), tta_transforms
).to(DEVICE)

# Set semua model TTA ke mode eval
tta_model_a.eval()
tta_model_b.eval()
tta_model_c.eval()
print("Semua model siap untuk TTA.")

# C.6. Siapkan Test Loader (tetap 518, TTA wrapper menangani sisanya)
final_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((518, 518)), # <-- Loader HANYA me-load gambar 518
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
test_df = convert_path_to_df(TEST_DIR, is_test=True)
test_dataset = TestDataset(test_df, 'Filepath', transform=final_test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# C.7. Fungsi bantu untuk ensemble (dengan OPSI WEIGHTS)
def make_submission(ensemble_name, prob_list, paths, weights=None):
    if weights is None:
        # Simple Averaging
        print(f"Membuat submit (Average): {ensemble_name}...")
        avg_probs = torch.stack(prob_list).mean(dim=0)
    else:
        # Weighted Averaging
        print(f"Membuat submit (Weighted): {ensemble_name}...")
        # Pastikan bobot dalam bentuk tensor yang tepat untuk broadcasting
        weights_tensor = torch.tensor(weights, dtype=torch.float32).view(-1, 1, 1)
        # Normalisasi bobot agar jumlahnya 1
        weights_tensor = weights_tensor / weights_tensor.sum() 
        
        weighted_probs = torch.stack(prob_list) * weights_tensor
        avg_probs = torch.sum(weighted_probs, dim=0)

    preds = torch.argmax(avg_probs, dim=1)
    submission_data = []
    
    # Ganti 'style' dengan 'label' jika diperlukan oleh format submisi
    # Saya menggunakan 'style' sesuai kode asli Anda
    for i, path in enumerate(paths):
        img_id = os.path.splitext(os.path.basename(path))[0]
        label_int = preds[i].item()
        label_str = reverse_label_mapping[label_int]
        submission_data.append({'id': img_id, 'style': label_str}) 
        
    df = pd.DataFrame(submission_data)
    df.sort_values(by='id', inplace=True)
    filename = f"submission_{ensemble_name}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ File {filename} berhasil dibuat!")

# C.8. Proses inference TTA
print("\nMembuat prediksi TTA ensemble dari ketiga model...")
all_probs_a, all_probs_b, all_probs_c, all_paths = [], [], [], []

with torch.no_grad():
    # Gunakan batch size lebih kecil jika OOM (Out of Memory)
    # TTA 8x augmentasi, jadi batch_size=8 akan memproses 64 gambar di VRAM
    for images, paths in tqdm(test_loader, desc="Inferensi Multi-Model TTA"):
        images_518 = images.to(DEVICE)

        # Semua model menerima input 518x518
        # TTA Wrapper akan menangani augmentasi
        # Wrapper kustom (Model224Wrapper) akan menangani resizing
        
        probs_a = tta_model_a(images_518)
        probs_b = tta_model_b(images_518)
        probs_c = tta_model_c(images_518)

        # Output dari tta.ClassificationTTAWrapper SUDAH di-softmax dan di-average
        
        # Simpan hasil batch
        all_probs_a.append(probs_a.cpu())
        all_probs_b.append(probs_b.cpu())
        all_probs_c.append(probs_c.cpu())
        all_paths.extend(paths)

# Gabungkan semua batch
all_probs_a = torch.cat(all_probs_a, dim=0)
all_probs_b = torch.cat(all_probs_b, dim=0)
all_probs_c = torch.cat(all_probs_c, dim=0)

# C.9. Buat berbagai kombinasi ensemble
print("\nMenyusun berbagai kombinasi ensemble...")

# --- Simple Averaging ---
make_submission("tta_ensemble_eva_swin", [all_probs_c, all_probs_b], all_paths)
make_submission("tta_ensemble_eva_fusion", [all_probs_c, all_probs_a], all_paths)
make_submission("tta_ensemble_swin_fusion", [all_probs_b, all_probs_a], all_paths)
make_submission("tta_ensemble_eva_swin_fusion", [all_probs_c, all_probs_b, all_probs_a], all_paths)


# C.10. Bersihkan memori
# del model_a_base, model_b_base, model_c_base
# del tta_model_a, tta_model_b, tta_model_c
# del all_probs_a, all_probs_b, all_probs_c, all_paths, test_loader, test_dataset
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (MULTI-MODEL TTA) ###
################################################################################

Mendefinisikan TTA transforms (Flip + Rotasi)...

Inisialisasi model dasar...


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Memuat bobot...
Memuat Model A (FusionDINOv2) dari: best_gradual_unfreeze_model_one.pth
Memuat Model B (SingleSwinModel) dari: best_swin_standalone.pth
Memuat Model C (SingleEVAModel) dari: EVA.pth

Membungkus model dengan TTA Wrapper...
 > Wrapper 224px dibuat
 > Wrapper 224px dibuat
Semua model siap untuk TTA.

Membuat prediksi TTA ensemble dari ketiga model...


Inferensi Multi-Model TTA:   0%|          | 0/258 [00:00<?, ?it/s]


Menyusun berbagai kombinasi ensemble...
Membuat submit (Average): tta_ensemble_eva_swin...
✅ File submission_tta_ensemble_eva_swin.csv berhasil dibuat!
Membuat submit (Average): tta_ensemble_eva_fusion...
✅ File submission_tta_ensemble_eva_fusion.csv berhasil dibuat!
Membuat submit (Average): tta_ensemble_swin_fusion...
✅ File submission_tta_ensemble_swin_fusion.csv berhasil dibuat!
Membuat submit (Average): tta_ensemble_eva_swin_fusion...
✅ File submission_tta_ensemble_eva_swin_fusion.csv berhasil dibuat!


In [ ]:
!pip install ttach

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ttach as tta  # <-- Impor TTA
import pandas as pd
import os
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score # <-- Perlu untuk validasi

# =================================================================================
# ASUMSI: Variabel dan Kelas ini sudah ada dari blok sebelumnya:
#
# - DEVICE = torch.device(...)
# - label_mapping = {...}
# - TEST_DIR = "..."
# - BATCH_SIZE, NUM_WORKERS, IMAGENET_MEAN, IMAGENET_STD
# - Class Definition: FusionDINOv2, SingleSwinModel, SingleEVAModel
# - Class Definition: SingleConvNextModel <--- (DARI BLOK B SEBELUMNYA)
# - Class Definition: CLAHETransform, TestDataset, CustomDataset <---
# - Fungsi Definition: convert_path_to_df
# - DataFrame: val_split <--- (PENTING: Pastikan ini tidak dihapus dari BLOK B)
# =================================================================================


####################################################################################
### BLOK C: FINAL ENSEMBLE INFERENCE & SUBMISSION (4-MODEL TTA + VALIDASI)
####################################################################################

print("\n" + "#"*80)
print("### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (4-MODEL TTA) ###")
print("#"*80 + "\n")

# C.1. Definisikan TTA transformations
print("Mendefinisikan TTA transforms (Flip + Rotasi)...")
tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 90, 180, 270]), 
    ]
)

# C.2. Buat Wrapper Model untuk resizing
class Model224Wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        print(" > Wrapper 224px dibuat")

    def forward(self, x_518):
        x_224 = F.interpolate(x_518, size=(224, 224), mode='bilinear', align_corners=False)
        return self.model(x_224)

# C.3. Inisialisasi model ASLI (tanpa TTA)
print("\nInisialisasi model dasar...")
model_a_base = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
model_b_base = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
model_c_base = SingleEVAModel(num_classes=len(label_mapping)).to(DEVICE)
model_d_base = SingleConvNextModel(num_classes=len(label_mapping)).to(DEVICE) # <-- TAMBAH

# C.4. Muat bobot ke model ASLI
print("Memuat bobot...")
print(f"Memuat Model A (FusionDINOv2) dari: best_gradual_unfreeze_model_one.pth")
model_a_base.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/4/best_gradual_unfreeze_model_one (1).pth", map_location=DEVICE))
print(f"Memuat Model B (SingleSwinModel) dari: best_swin_standalone.pth")
model_b_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_swin_standalone (3).pth", map_location=DEVICE))
print(f"Memuat Model C (SingleEVAModel) dari: EVA.pth")
model_c_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_eva_standalone (3).pth", map_location=DEVICE))
print(f"Memuat Model D (SingleConvNextModel) dari: best_convnext_standalone.pth") # <-- TAMBAH
model_d_base.load_state_dict(torch.load("best_convnext_standalone.pth", map_location=DEVICE)) # <-- TAMBAH

# C.5. BUNGKUS model dengan TTA
print("\nMembungkus model dengan TTA Wrapper...")
# Model A (Fusion/DINO) menggunakan 518px, bisa langsung dibungkus
tta_model_a = tta.ClassificationTTAWrapper(model_a_base, tta_transforms).to(DEVICE)

# Model B (Swin), C (EVA), dan D (ConvNext) pakai wrapper 224px
tta_model_b = tta.ClassificationTTAWrapper(
    Model224Wrapper(model_b_base), tta_transforms
).to(DEVICE)

tta_model_c = tta.ClassificationTTAWrapper(
    Model224Wrapper(model_c_base), tta_transforms
).to(DEVICE)

tta_model_d = tta.ClassificationTTAWrapper( # <-- TAMBAH
    Model224Wrapper(model_d_base), tta_transforms
).to(DEVICE)

# Set semua model TTA ke mode eval
tta_model_a.eval()
tta_model_b.eval()
tta_model_c.eval()
tta_model_d.eval() # <-- TAMBAH
print("Semua 4 model siap untuk TTA.")

# C.6. Transformasi Test (akan digunakan untuk Val dan Test)
final_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((518, 518)), # <-- Loader HANYA me-load gambar 518
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# =============================================================================
# === C.VAL: VALIDASI ENSEMBLE PADA VALIDATION SET (BLOK BARU)
# =============================================================================
print("\n" + "="*50)
print("### MENJALANKAN VALIDASI ENSEMBLE PADA VAL SET ###")
print("="*50)

try:
    # 1. Buat Val Dataset & Loader (menggunakan transform 518px)
    # Pastikan 'val_split' (DataFrame) dan 'CustomDataset' (Class) ada
    val_dataset_tta = CustomDataset(
        val_split, 
        'Filepath', 
        'Label', 
        final_test_transform # <-- Pakai transform 518px
    )
    val_loader_tta = DataLoader(
        val_dataset_tta, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=NUM_WORKERS
    )

    all_val_probs_a, all_val_probs_b, all_val_probs_c, all_val_probs_d = [], [], [], []
    all_val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader_tta, desc="Validasi TTA Ensemble"):
            images_518 = images.to(DEVICE)
            
            # Dapatkan prediksi TTA
            probs_a = tta_model_a(images_518)
            probs_b = tta_model_b(images_518)
            probs_c = tta_model_c(images_518)
            probs_d = tta_model_d(images_518) # <-- TAMBAH

            # Simpan hasil
            all_val_probs_a.append(probs_a.cpu())
            all_val_probs_b.append(probs_b.cpu())
            all_val_probs_c.append(probs_c.cpu())
            all_val_probs_d.append(probs_d.cpu()) # <-- TAMBAH
            all_val_labels.append(labels.cpu())

    # Gabungkan batch
    all_val_probs_a = torch.cat(all_val_probs_a, dim=0)
    all_val_probs_b = torch.cat(all_val_probs_b, dim=0)
    all_val_probs_c = torch.cat(all_val_probs_c, dim=0)
    all_val_probs_d = torch.cat(all_val_probs_d, dim=0) # <-- TAMBAH
    all_val_labels = torch.cat(all_val_labels, dim=0)

    # 2. Hitung Metrik Ensemble (Contoh: Weighted Averaging)
    # !!! GANTI BOBOT INI dengan skor Akurasi/F1 validasi Anda !!!
    W_A_FUSION = 0.93 # Ganti dengan skor validasi asli Anda
    W_B_SWIN = 0.93   # Ganti dengan skor validasi asli Anda
    W_C_EVA = 0.938   # Ganti dengan skor validasi asli Anda
    W_D_CONVNEXT = 0.94 # Ganti dengan skor validasi asli Anda

    weights = torch.tensor(
        [W_A_FUSION, W_B_SWIN, W_C_EVA, W_D_CONVNEXT], 
        dtype=torch.float32
    ).view(-1, 1, 1)
    weights = weights / weights.sum() # Normalisasi

    prob_list = [all_val_probs_a, all_val_probs_b, all_val_probs_c, all_val_probs_d]
    weighted_probs = torch.stack(prob_list) * weights
    avg_probs = torch.sum(weighted_probs, dim=0)

    final_preds = torch.argmax(avg_probs, dim=1).numpy()
    true_labels = all_val_labels.numpy()

    # 3. Tampilkan Hasil
    val_f1 = f1_score(true_labels, final_preds, average="macro")
    val_acc = accuracy_score(true_labels, final_preds)

    print("\n--- HASIL VALIDASI TTA ENSEMBLE (4 MODEL) ---")
    print(f"📈 F1-Score Macro: {val_f1:.6f}")
    print(f"🎯 Accuracy:       {val_acc:.6f}")
    print("----------------------------------------------\n")
    
except NameError as e:
    print(f"❌ GAGAL: Tidak bisa menjalankan validasi.")
    print(f"Error: {e}. (Apakah 'val_split' atau 'CustomDataset' terdefinisi?)")
    print("Melanjutkan ke inferensi test set...\n")


# =============================================================================
# === C.TEST: INFERENSI PADA TEST SET
# =============================================================================
print("\n" + "="*50)
print("### MENJALANKAN INFERENSI PADA TEST SET ###")
print("="*50)

# C.7. Siapkan Test Loader (menggunakan final_test_transform 518px)
test_df = convert_path_to_df(TEST_DIR, is_test=True)
test_dataset = TestDataset(test_df, 'Filepath', transform=final_test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# C.8. Fungsi bantu untuk ensemble (dengan OPSI WEIGHTS)
def make_submission(ensemble_name, prob_list, paths, weights=None):
    if weights is None:
        # Simple Averaging
        print(f"Membuat submit (Average): {ensemble_name}...")
        avg_probs = torch.stack(prob_list).mean(dim=0)
    else:
        # Weighted Averaging
        print(f"Membuat submit (Weighted): {ensemble_name}...")
        weights_tensor = torch.tensor(weights, dtype=torch.float32).view(-1, 1, 1)
        weights_tensor = weights_tensor / weights_tensor.sum() 
        
        weighted_probs = torch.stack(prob_list) * weights_tensor
        avg_probs = torch.sum(weighted_probs, dim=0)

    preds = torch.argmax(avg_probs, dim=1)
    submission_data = []
    
    for i, path in enumerate(paths):
        img_id = os.path.splitext(os.path.basename(path))[0]
        label_int = preds[i].item()
        label_str = reverse_label_mapping[label_int]
        # Pastikan nama kolom 'id' dan 'label' (atau 'style') sesuai format submisi
        submission_data.append({'id': img_id, 'label': label_str}) 
        
    df = pd.DataFrame(submission_data)
    df.sort_values(by='id', inplace=True)
    filename = f"submission_{ensemble_name}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ File {filename} berhasil dibuat!")

# C.9. Proses inference TTA
print("\nMembuat prediksi TTA ensemble dari 4 model...")
all_probs_a, all_probs_b, all_probs_c, all_probs_d, all_paths = [], [], [], [], [] # <-- TAMBAH

with torch.no_grad():
    for images, paths in tqdm(test_loader, desc="Inferensi Multi-Model TTA"):
        images_518 = images.to(DEVICE)
        
        probs_a = tta_model_a(images_518)
        probs_b = tta_model_b(images_518)
        probs_c = tta_model_c(images_518)
        probs_d = tta_model_d(images_518) # <-- TAMBAH

        # Simpan hasil batch
        all_probs_a.append(probs_a.cpu())
        all_probs_b.append(probs_b.cpu())
        all_probs_c.append(probs_c.cpu())
        all_probs_d.append(probs_d.cpu()) # <-- TAMBAH
        all_paths.extend(paths)

# Gabungkan semua batch
all_probs_a = torch.cat(all_probs_a, dim=0)
all_probs_b = torch.cat(all_probs_b, dim=0)
all_probs_c = torch.cat(all_probs_c, dim=0)
all_probs_d = torch.cat(all_probs_d, dim=0) # <-- TAMBAH

# C.10. Buat berbagai kombinasi ensemble
print("\nMenyusun berbagai kombinasi ensemble...")

# --- Simple Averaging (4 Model) ---
make_submission(
    "tta_ensemble_4_models_avg", 
    [all_probs_a, all_probs_b, all_probs_c, all_probs_d], 
    all_paths
)

# --- Weighted Averaging (4 Model) ---
# Gunakan bobot yang SAMA seperti di blok validasi
make_submission(
    "tta_ensemble_4_models_weighted", 
    [all_probs_a, all_probs_b, all_probs_c, all_probs_d], 
    all_paths,
    weights=[W_A_FUSION, W_B_SWIN, W_C_EVA, W_D_CONVNEXT]
)

print("\n" + "="*50 + "\nSELURUH PROSES ENSEMBLE TTA SELESAI\n" + "="*50)

# # C.11. Bersihkan memori
# del model_a_base, model_b_base, model_c_base, model_d_base
# del tta_model_a, tta_model_b, tta_model_c, tta_model_d
# del all_probs_a, all_probs_b, all_probs_c, all_probs_d, all_paths, test_loader, test_dataset
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (4-MODEL TTA) ###
################################################################################

Mendefinisikan TTA transforms (Flip + Rotasi)...

Inisialisasi model dasar...


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Memuat bobot...
Memuat Model A (FusionDINOv2) dari: best_gradual_unfreeze_model_one.pth
Memuat Model B (SingleSwinModel) dari: best_swin_standalone.pth
Memuat Model C (SingleEVAModel) dari: EVA.pth
Memuat Model D (SingleConvNextModel) dari: best_convnext_standalone.pth

Membungkus model dengan TTA Wrapper...
 > Wrapper 224px dibuat
 > Wrapper 224px dibuat
 > Wrapper 224px dibuat
Semua 4 model siap untuk TTA.

### MENJALANKAN VALIDASI ENSEMBLE PADA VAL SET ###


Validasi TTA Ensemble:   0%|          | 0/50 [00:00<?, ?it/s]


--- HASIL VALIDASI TTA ENSEMBLE (4 MODEL) ---
📈 F1-Score Macro: 0.967073
🎯 Accuracy:       0.967296
----------------------------------------------


### MENJALANKAN INFERENSI PADA TEST SET ###

Membuat prediksi TTA ensemble dari 4 model...


Inferensi Multi-Model TTA:   0%|          | 0/129 [00:00<?, ?it/s]


Menyusun berbagai kombinasi ensemble...
Membuat submit (Average): tta_ensemble_4_models_avg...
✅ File submission_tta_ensemble_4_models_avg.csv berhasil dibuat!
Membuat submit (Weighted): tta_ensemble_4_models_weighted...
✅ File submission_tta_ensemble_4_models_weighted.csv berhasil dibuat!

SELURUH PROSES ENSEMBLE TTA SELESAI


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import ttach as tta
import pandas as pd
import os
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score

# =================================================================================
# ASUMSI: Semua variabel, kelas (termasuk SingleConvNextModel), 
# dan DataFrame (val_split) sudah ada dari blok sebelumnya.
# =================================================================================


####################################################################################
### BLOK C: FINAL ENSEMBLE INFERENCE & SUBMISSION (PERBANDINGAN KOMBINASI LENGKAP)
####################################################################################

print("\n" + "#"*80)
print("### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (4-MODEL TTA) ###")
print("#"*80 + "\n")

# C.1. Definisikan TTA transformations
print("Mendefinisikan TTA transforms (Flip + Rotasi)...")
tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 90, 180, 270]), 
    ]
)

# C.2. Buat Wrapper Model untuk resizing
class Model224Wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        print(" > Wrapper 224px dibuat")

    def forward(self, x_518):
        x_224 = F.interpolate(x_518, size=(224, 224), mode='bilinear', align_corners=False)
        return self.model(x_224)

# C.3. Inisialisasi model ASLI
print("\nInisialisasi model dasar...")
model_a_base = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
model_b_base = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
model_c_base = SingleEVAModel(num_classes=len(label_mapping)).to(DEVICE)
model_d_base = SingleConvNextModel(num_classes=len(label_mapping)).to(DEVICE)

# C.4. Muat bobot ke model ASLI
print("Memuat bobot...")
print(f"Memuat Model A (FusionDINOv2)")
model_a_base.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/4/best_gradual_unfreeze_model_one (1).pth", map_location=DEVICE))
print(f"Memuat Model B (SingleSwinModel)")
model_b_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_swin_standalone (3).pth", map_location=DEVICE))
print(f"Memuat Model C (SingleEVAModel)")
model_c_base.load_state_dict(torch.load("/kaggle/input/palingbaru/pytorch/default/2/best_eva_standalone (3).pth", map_location=DEVICE))
print(f"Memuat Model D (SingleConvNextModel)")
model_d_base.load_state_dict(torch.load("best_convnext_standalone.pth", map_location=DEVICE))

# C.5. BUNGKUS model dengan TTA
print("\nMembungkus model dengan TTA Wrapper...")
tta_model_a = tta.ClassificationTTAWrapper(model_a_base, tta_transforms).to(DEVICE)
tta_model_b = tta.ClassificationTTAWrapper(Model224Wrapper(model_b_base), tta_transforms).to(DEVICE)
tta_model_c = tta.ClassificationTTAWrapper(Model224Wrapper(model_c_base), tta_transforms).to(DEVICE)
tta_model_d = tta.ClassificationTTAWrapper(Model224Wrapper(model_d_base), tta_transforms).to(DEVICE)

# Set semua model TTA ke mode eval
tta_model_a.eval()
tta_model_b.eval()
tta_model_c.eval()
tta_model_d.eval()
print("Semua 4 model siap untuk TTA.")

# C.6. Transformasi Test
final_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((518, 518)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# =============================================================================
# === C.VAL: VALIDASI ENSEMBLE PADA VALIDATION SET (BLOK BARU)
# =============================================================================
print("\n" + "="*50)
print("### MENJALANKAN VALIDASI ENSEMBLE PADA VAL SET ###")
print("="*50)

# C.VAL.1. Fungsi Helper untuk Evaluasi
def evaluate_ensemble(name, prob_list, true_labels_np):
    """Menghitung F1 & Akurasi untuk daftar probabilitas (simple averaging)"""
    avg_probs = torch.stack(prob_list).mean(dim=0)
    final_preds = torch.argmax(avg_probs, dim=1).numpy()
    
    f1 = f1_score(true_labels_np, final_preds, average="macro")
    acc = accuracy_score(true_labels_np, final_preds)
    
    print(f"| {name.ljust(30)} | {f1:.6f} | {acc:.6f} |")
    return (name, f1, acc)

try:
    # C.VAL.2. Buat Val Loader
    val_dataset_tta = CustomDataset(val_split, 'Filepath', 'Label', final_test_transform)
    val_loader_tta = DataLoader(val_dataset_tta, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    all_val_probs_a, all_val_probs_b, all_val_probs_c, all_val_probs_d = [], [], [], []
    all_val_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader_tta, desc="Validasi TTA Ensemble"):
            images_518 = images.to(DEVICE)
            
            probs_a = tta_model_a(images_518)
            probs_b = tta_model_b(images_518)
            probs_c = tta_model_c(images_518)
            probs_d = tta_model_d(images_518)

            all_val_probs_a.append(probs_a.cpu())
            all_val_probs_b.append(probs_b.cpu())
            all_val_probs_c.append(probs_c.cpu())
            all_val_probs_d.append(probs_d.cpu())
            all_val_labels.append(labels.cpu())

    # C.VAL.3. Gabungkan Batch
    all_val_probs_a = torch.cat(all_val_probs_a, dim=0)
    all_val_probs_b = torch.cat(all_val_probs_b, dim=0)
    all_val_probs_c = torch.cat(all_val_probs_c, dim=0)
    all_val_probs_d = torch.cat(all_val_probs_d, dim=0)
    all_val_labels_np = torch.cat(all_val_labels, dim=0).numpy()

    # C.VAL.4. Tampilkan Tabel Perbandingan (Simple Averaging)
    print("\n--- HASIL VALIDASI TTA ENSEMBLE (SIMPLE AVERAGING) ---")
    print("| Model Combination                | F1-Macro | Accuracy |")
    print("|----------------------------------|----------|----------|")
    
    results = [] # Simpan hasil untuk diurutkan
    
    # Model Tunggal (untuk baseline)
    results.append(evaluate_ensemble("Model A: DINO", [all_val_probs_a], all_val_labels_np))
    results.append(evaluate_ensemble("Model B: Swin", [all_val_probs_b], all_val_labels_np))
    results.append(evaluate_ensemble("Model C: EVA", [all_val_probs_c], all_val_labels_np))
    results.append(evaluate_ensemble("Model D: ConvNext", [all_val_probs_d], all_val_labels_np))
    print("|----------------------------------|----------|----------|")

    # Kombinasi 2 Model
    results.append(evaluate_ensemble("A+B (DINO+Swin)", [all_val_probs_a, all_val_probs_b], all_val_labels_np))
    results.append(evaluate_ensemble("A+C (DINO+EVA)", [all_val_probs_a, all_val_probs_c], all_val_labels_np))
    results.append(evaluate_ensemble("A+D (DINO+ConvNext)", [all_val_probs_a, all_val_probs_d], all_val_labels_np))
    results.append(evaluate_ensemble("B+C (Swin+EVA)", [all_val_probs_b, all_val_probs_c], all_val_labels_np))
    results.append(evaluate_ensemble("B+D (Swin+ConvNext)", [all_val_probs_b, all_val_probs_d], all_val_labels_np))
    results.append(evaluate_ensemble("C+D (EVA+ConvNext)", [all_val_probs_c, all_val_probs_d], all_val_labels_np))
    print("|----------------------------------|----------|----------|")

    # Kombinasi 3 Model
    results.append(evaluate_ensemble("A+B+C (DINO+Swin+EVA)", [all_val_probs_a, all_val_probs_b, all_val_probs_c], all_val_labels_np))
    results.append(evaluate_ensemble("A+B+D (DINO+Swin+ConvNext)", [all_val_probs_a, all_val_probs_b, all_val_probs_d], all_val_labels_np))
    results.append(evaluate_ensemble("A+C+D (DINO+EVA+ConvNext)", [all_val_probs_a, all_val_probs_c, all_val_probs_d], all_val_labels_np))
    results.append(evaluate_ensemble("B+C+D (Swin+EVA+ConvNext)", [all_val_probs_b, all_val_probs_c, all_val_probs_d], all_val_labels_np))
    print("|----------------------------------|----------|----------|")

    # Kombinasi 4 Model
    results.append(evaluate_ensemble("A+B+C+D (ALL 4 AVG)", [all_val_probs_a, all_val_probs_b, all_val_probs_c, all_val_probs_d], all_val_labels_np))
    print("-----------------------------------------------------")

    # Temukan yang terbaik
    best_result = sorted(results, key=lambda x: x[2], reverse=True)[0]
    print(f"\n🏆 Kombinasi Terbaik (Simple Avg): {best_result[0]} (Acc: {best_result[2]:.6f})")
    
except NameError as e:
    print(f"❌ GAGAL: Tidak bisa menjalankan validasi. Error: {e}")
    print("Melanjutkan ke inferensi test set...\n")


# =============================================================================
# === C.TEST: INFERENSI PADA TEST SET
# =============================================================================
print("\n" + "="*50)
print("### MENJALANKAN INFERENSI PADA TEST SET ###")
print("="*50)

# C.7. Siapkan Test Loader
test_df = convert_path_to_df(TEST_DIR, is_test=True)
test_dataset = TestDataset(test_df, 'Filepath', transform=final_test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

# C.8. Fungsi bantu untuk ensemble (dengan OPSI WEIGHTS)
def make_submission(ensemble_name, prob_list, paths, weights=None):
    if weights is None:
        print(f"Membuat submit (Average): {ensemble_name}...")
        avg_probs = torch.stack(prob_list).mean(dim=0)
    else:
        print(f"Membuat submit (Weighted): {ensemble_name}...")
        weights_tensor = torch.tensor(weights, dtype=torch.float32).view(-1, 1, 1)
        weights_tensor = weights_tensor / weights_tensor.sum() 
        weighted_probs = torch.stack(prob_list) * weights_tensor
        avg_probs = torch.sum(weighted_probs, dim=0)

    preds = torch.argmax(avg_probs, dim=1)
    submission_data = []
    
    for i, path in enumerate(paths):
        img_id = os.path.splitext(os.path.basename(path))[0]
        label_int = preds[i].item()
        label_str = reverse_label_mapping[label_int]
        submission_data.append({'id': img_id, 'label': label_str}) # Pastikan nama kolom 'label'
        
    df = pd.DataFrame(submission_data)
    df.sort_values(by='id', inplace=True)
    filename = f"submission_{ensemble_name}.csv"
    df.to_csv(filename, index=False)
    print(f"✅ File {filename} berhasil dibuat!")

# C.9. Proses inference TTA
print("\nMembuat prediksi TTA ensemble dari 4 model...")
all_probs_a, all_probs_b, all_probs_c, all_probs_d, all_paths = [], [], [], [], []

with torch.no_grad():
    for images, paths in tqdm(test_loader, desc="Inferensi Multi-Model TTA"):
        images_518 = images.to(DEVICE)
        
        probs_a = tta_model_a(images_518)
        probs_b = tta_model_b(images_518)
        probs_c = tta_model_c(images_518)
        probs_d = tta_model_d(images_518) 

        all_probs_a.append(probs_a.cpu())
        all_probs_b.append(probs_b.cpu())
        all_probs_c.append(probs_c.cpu())
        all_probs_d.append(probs_d.cpu())
        all_paths.extend(paths)

# Gabungkan semua batch
all_probs_a = torch.cat(all_probs_a, dim=0)
all_probs_b = torch.cat(all_probs_b, dim=0)
all_probs_c = torch.cat(all_probs_c, dim=0)
all_probs_d = torch.cat(all_probs_d, dim=0)

# C.10. Buat SEMUA kombinasi submisi (Simple Averaging)
print("\nMenyusun berbagai kombinasi ensemble untuk submisi...")

# --- Kombinasi 2 Model ---
make_submission("tta_A+B", [all_probs_a, all_probs_b], all_paths)
make_submission("tta_A+C", [all_probs_a, all_probs_c], all_paths)
make_submission("tta_A+D", [all_probs_a, all_probs_d], all_paths)
make_submission("tta_B+C", [all_probs_b, all_probs_c], all_paths)
make_submission("tta_B+D", [all_probs_b, all_probs_d], all_paths)
make_submission("tta_C+D", [all_probs_c, all_probs_d], all_paths)

# --- Kombinasi 3 Model ---
make_submission("tta_A+B+C", [all_probs_a, all_probs_b, all_probs_c], all_paths)
make_submission("tta_A+B+D", [all_probs_a, all_probs_b, all_probs_d], all_paths)
make_submission("tta_A+C+D", [all_probs_a, all_probs_c, all_probs_d], all_paths)
make_submission("tta_B+C+D", [all_probs_b, all_probs_c, all_probs_d], all_paths)

# --- Kombinasi 4 Model ---
make_submission("tta_A+B+C+D", [all_probs_a, all_probs_b, all_probs_c, all_probs_d], all_paths)

# --- (Opsional) Weighted Average Berdasarkan Skor Val Anda ---
# Ganti dengan skor akurasi validasi individu
# W_A = 0.93; W_B = 0.93; W_C = 0.938; W_D = 0.94 
# make_submission(
#     "tta_4_models_WEIGHTED", 
#     [all_probs_a, all_probs_b, all_probs_c, all_probs_d], 
#     all_paths,
#     weights=[W_A, W_B, W_C, W_D]
# )

print("\n" + "="*50 + "\nSELURUH PROSES ENSEMBLE TTA SELESAI\n" + "="*50)

# # C.11. Bersihkan memori
# del model_a_base, model_b_base, model_c_base, model_d_base
# del tta_model_a, tta_model_b, tta_model_c, tta_model_d
# del all_probs_a, all_probs_b, all_probs_c, all_probs_d, all_paths, test_loader, test_dataset
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: ENSEMBLE INFERENCE & SUBMISSION (4-MODEL TTA) ###
################################################################################

Mendefinisikan TTA transforms (Flip + Rotasi)...

Inisialisasi model dasar...
Memuat bobot...
Memuat Model A (FusionDINOv2)
Memuat Model B (SingleSwinModel)
Memuat Model C (SingleEVAModel)
Memuat Model D (SingleConvNextModel)

Membungkus model dengan TTA Wrapper...
 > Wrapper 224px dibuat
 > Wrapper 224px dibuat
 > Wrapper 224px dibuat
Semua 4 model siap untuk TTA.

### MENJALANKAN VALIDASI ENSEMBLE PADA VAL SET ###


Validasi TTA Ensemble:   0%|          | 0/50 [00:00<?, ?it/s]


--- HASIL VALIDASI TTA ENSEMBLE (SIMPLE AVERAGING) ---
| Model Combination                | F1-Macro | Accuracy |
|----------------------------------|----------|----------|
| Model A: DINO                  | 0.968612 | 0.968553 |
| Model B: Swin                  | 0.924038 | 0.924528 |
| Model C: EVA                   | 0.938813 | 0.939623 |
| Model D: ConvNext              | 0.905987 | 0.906918 |
|----------------------------------|----------|----------|
| A+B (DINO+Swin)                | 0.972218 | 0.972327 |
| A+C (DINO+EVA)                 | 0.973575 | 0.973585 |
| A+D (DINO+ConvNext)            | 0.973415 | 0.973585 |
| B+C (Swin+EVA)                 | 0.941810 | 0.942138 |
| B+D (Swin+ConvNext)            | 0.930334 | 0.930818 |
| C+D (EVA+ConvNext)             | 0.939072 | 0.939623 |
|----------------------------------|----------|----------|
| A+B+C (DINO+Swin+EVA)          | 0.972223 | 0.972327 |
| A+B+D (DINO+Swin+ConvNext)     | 0.970890 | 0.971069 |
| A+C+D (DINO+EVA+ConvNe

Inferensi Multi-Model TTA:   0%|          | 0/129 [00:00<?, ?it/s]


Menyusun berbagai kombinasi ensemble untuk submisi...
Membuat submit (Average): tta_A+B...
✅ File submission_tta_A+B.csv berhasil dibuat!
Membuat submit (Average): tta_A+C...
✅ File submission_tta_A+C.csv berhasil dibuat!
Membuat submit (Average): tta_A+D...
✅ File submission_tta_A+D.csv berhasil dibuat!
Membuat submit (Average): tta_B+C...
✅ File submission_tta_B+C.csv berhasil dibuat!
Membuat submit (Average): tta_B+D...
✅ File submission_tta_B+D.csv berhasil dibuat!
Membuat submit (Average): tta_C+D...
✅ File submission_tta_C+D.csv berhasil dibuat!
Membuat submit (Average): tta_A+B+C...
✅ File submission_tta_A+B+C.csv berhasil dibuat!
Membuat submit (Average): tta_A+B+D...
✅ File submission_tta_A+B+D.csv berhasil dibuat!
Membuat submit (Average): tta_A+C+D...
✅ File submission_tta_A+C+D.csv berhasil dibuat!
Membuat submit (Average): tta_B+C+D...
✅ File submission_tta_B+C+D.csv berhasil dibuat!
Membuat submit (Average): tta_A+B+C+D...
✅ File submission_tta_A+B+C+D.csv berhasil dibua

In [ ]:
import torch
import torch.nn as nn
import ttach as tta
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm
import torch.nn.functional as F

####################################################################################
### BLOK VALIDASI KHUSUS: Model Fusion (DINOv2) TTA vs Non-TTA
####################################################################################

print("\n" + "#"*80)
print("### BLOK VALIDASI KHUSUS: Model Fusion (DINOv2) ###")
print("#"*80 + "\n")

# 1. Inisialisasi Model (Asumsi Kelas sudah terdefinisi)
print("Memuat Model A (FusionDINOv2)...")
try:
    model_a_base = FusionDINOv2(num_classes=len(label_mapping)).to(DEVICE)
    model_a_base.load_state_dict(torch.load("/kaggle/input/swim/pytorch/default/4/best_gradual_unfreeze_model_one (1).pth", map_location=DEVICE))
    model_a_base.eval() # Set ke mode eval

    # 2. Inisialisasi Model TTA
    # (Asumsi tta_transforms sudah terdefinisi dari BLOK C)
    print("Membungkus Model A dengan TTA...")
    tta_model_a = tta.ClassificationTTAWrapper(model_a_base, tta_transforms).to(DEVICE)
    tta_model_a.eval() # Set ke mode eval

    # 3. Buat Val Loader
    # (Asumsi val_split (DataFrame) dan CustomDataset (Class) sudah ada)
    # (Asumsi final_test_transform (518px) sudah ada)
    print(f"Mempersiapkan Validation Loader (data: {len(val_split)})...")
    val_dataset_check = CustomDataset(
        val_split, 
        'Filepath', 
        'Label', 
        final_test_transform # <-- Pakai transform 518px
    )
    val_loader_check = DataLoader(
        val_dataset_check, 
        batch_size=BATCH_SIZE, # Atur BATCH_SIZE lebih kecil jika OOM
        shuffle=False, 
        num_workers=NUM_WORKERS
    )

    # 4. Jalankan Evaluasi
    all_labels_list = []
    all_preds_no_tta_list = []
    all_preds_tta_list = []

    print("Memulai evaluasi TTA vs Non-TTA...")
    with torch.no_grad():
        for images, labels in tqdm(val_loader_check, desc="Validasi Model A"):
            images_518 = images.to(DEVICE)
            labels_cpu = labels.cpu()
            all_labels_list.append(labels_cpu)

            # --- 1. Prediksi NON-TTA (Manual) ---
            with torch.amp.autocast(device_type = 'cuda'):
                outputs_no_tta = model_a_base(images_518)
            preds_no_tta = torch.argmax(outputs_no_tta, dim=1).cpu()
            all_preds_no_tta_list.append(preds_no_tta)

            # --- 2. Prediksi DENGAN TTA ---
            # tta_model_a sudah menangani autocast, softmax, dan averaging
            probs_tta = tta_model_a(images_518) 
            preds_tta = torch.argmax(probs_tta, dim=1).cpu()
            all_preds_tta_list.append(preds_tta)

    # 5. Gabungkan hasil
    all_labels = torch.cat(all_labels_list).numpy()
    all_preds_no_tta = torch.cat(all_preds_no_tta_list).numpy()
    all_preds_tta = torch.cat(all_preds_tta_list).numpy()

    # 6. Hitung dan Tampilkan Metrik
    print("\n--- HASIL VALIDASI UNTUK Model A (FusionDINOv2) ---")
    
    # Non-TTA
    f1_no_tta = f1_score(all_labels, all_preds_no_tta, average="macro")
    acc_no_tta = accuracy_score(all_labels, all_preds_no_tta)
    print("\nMetrik TANPA TTA:")
    print(f"  > F1-Score Macro: {f1_no_tta:.6f}")
    print(f"  > Accuracy:       {acc_no_tta:.6f}")

    # TTA
    f1_tta = f1_score(all_labels, all_preds_tta, average="macro")
    acc_tta = accuracy_score(all_labels, all_preds_tta)
    print("\nMetrik DENGAN TTA (Flip + Rotasi):")
    print(f"  > F1-Score Macro: {f1_tta:.6f}")
    print(f"  > Accuracy:       {acc_tta:.6f}")
    
    print("\n" + "="*50)
    print(f"📈 PENINGKATAN (TTA - NonTTA):")
    print(f"   F1-Score: {(f1_tta - f1_no_tta):+.6f}")
    print(f"   Accuracy: {(acc_tta - acc_no_tta):+.6f}")
    print("="*50)

    # 7. Bersihkan Memori
    print("Membersihkan memori...")
    del model_a_base, tta_model_a, val_loader_check, val_dataset_check
    del all_labels, all_preds_no_tta, all_preds_tta
    torch.cuda.empty_cache()

except NameError as e:
    print(f"\n❌ GAGAL: Tidak bisa menjalankan validasi.")
    print(f"Error: {e}.")
    print("Pastikan 'val_split', 'CustomDataset', 'FusionDINOv2', 'tta_transforms',")
    print("dan 'final_test_transform' sudah terdefinisi.")


################################################################################
### BLOK VALIDASI KHUSUS: Model Fusion (DINOv2) ###
################################################################################

Memuat Model A (FusionDINOv2)...
Membungkus Model A dengan TTA...
Mempersiapkan Validation Loader (data: 795)...
Memulai evaluasi TTA vs Non-TTA...


Validasi Model A:   0%|          | 0/50 [00:00<?, ?it/s]


--- HASIL VALIDASI UNTUK Model A (FusionDINOv2) ---

Metrik TANPA TTA:
  > F1-Score Macro: 0.963352
  > Accuracy:       0.963522

Metrik DENGAN TTA (Flip + Rotasi):
  > F1-Score Macro: 0.968612
  > Accuracy:       0.968553

📈 PENINGKATAN (TTA - NonTTA):
   F1-Score: +0.005260
   Accuracy: +0.005031
Membersihkan memori...


In [ ]:
# GANTI NAMA FILE BOBOT
BEST_MODEL_NAME_EVA = "best_eva_standalone.pth"
# (Asumsi idx_to_class, TEST_DIR, test_transform, dll sudah terdefinisi)
# ...

print("\n" + "#"*80)
print("### MEMULAI BLOK C: INFERENSI DAN SUBMISI EVA TRANSFORMER ###") # Ganti teks
print("#"*80 + "\n")

# C.1. Definisikan Custom Dataset untuk Data Tes
# (Tidak ada perubahan di sini, TestImageDataset sudah benar)
class TestImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.image_files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        self.transform = transform
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

# C.2. Tentukan Transformasi untuk Data Tes
# (Tidak ada perubahan di sini, test_transform sudah benar)
test_transform = transforms.Compose([
    CLAHETransform(), 
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# C.3. Inisialisasi Dataset dan DataLoader
# (Tidak ada perubahan di sini)
test_dataset = TestImageDataset(TEST_DIR, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) 

# C.4. Inisialisasi Model dan Pemuatan Bobot (DIGANTI KE EVA)
model_inferensi = SingleEvaModel(num_classes=len(label_mapping)).to(DEVICE) # Panggil kelas EVA
print(f"Memuat bobot dari: {BEST_MODEL_NAME_EVA}") # Ganti variabel

if os.path.exists(BEST_MODEL_NAME_EVA): # Ganti variabel
    model_inferensi.load_state_dict(torch.load(BEST_MODEL_NAME_EVA, map_location=DEVICE)) # Ganti variabel
    print("Bobot EVA Transformer berhasil dimuat.") # Ganti teks
else:
    print(f"❌ Error: File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan!") # Ganti variabel
    # raise FileNotFoundError(f"File bobot {BEST_MODEL_NAME_EVA} tidak ditemukan.")


# C.5. Melakukan Inferensi
# (Tidak ada perubahan di sini)
model_inferensi.eval() 
image_ids = []
predictions = []

print("Memulai inferensi pada data tes...")
with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Inferensi"):
        images = images.to(DEVICE)
        with torch.amp.autocast(device_type = 'cuda' if 'cuda' in DEVICE.type else 'cpu'):
            outputs = model_inferensi(images)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        image_ids.extend(file_names)
        predictions.extend(preds)

print("Inferensi selesai. Total prediksi:", len(predictions))

# C.6. Pemetaan Ulang Label ke Nama Kelas
# (Tidak ada perubahan di sini)
# Buat reverse mapping: indeks ke nama kelas
idx_to_class = {v: k for k, v in label_mapping.items()}
predicted_labels = [idx_to_class[p] for p in predictions]

# C.7. Membuat DataFrame Submisi
# (Tidak ada perubahan di sini)
submission_df = pd.DataFrame({
    'id': image_ids,
    'label': predicted_labels
})

# C.8. Menyimpan ke File CSV (GANTI NAMA FILE)
submission_filename = "submission_eva_transformer.csv" 
submission_df.to_csv(submission_filename, index=False)

print("\n" + "="*50)
print(f"🎉 Submisi berhasil dibuat!")
print(f"File submisi disimpan di: {submission_filename}")
print("Contoh 5 baris pertama:\n", submission_df.head())
print("="*50)

# C.9. Bersihkan Memori GPU
print("Membersihkan memori GPU setelah inferensi...")
# del model_inferensi
# torch.cuda.empty_cache()


################################################################################
### MEMULAI BLOK C: INFERENSI DAN SUBMISI EVA TRANSFORMER ###
################################################################################

Memuat bobot dari: best_eva_standalone.pth
Bobot EVA Transformer berhasil dimuat.
Memulai inferensi pada data tes...


Inferensi:   0%|          | 0/129 [00:00<?, ?it/s]

Inferensi selesai. Total prediksi: 2057

🎉 Submisi berhasil dibuat!
File submisi disimpan di: submission_eva_transformer.csv
Contoh 5 baris pertama:
          id        label
0  0001.jpg  Sate Padang
1  0002.jpg      Rendang
2  0003.jpg  Ayam Betutu
3  0004.jpg      Rendang
4  0005.jpg     Ayam Pop
Membersihkan memori GPU setelah inferensi...
